In [1]:
import pandas as pd
import numpy as np

### Review Topic Composition File

In [2]:
import ast
with open('reviews-zomato-dubai-topic-composition-spark.txt') as f:
    data = ast.literal_eval(f.read())

In [3]:
df = pd.DataFrame(data)
print(df.shape)
df.head()

(8847, 2)


,id,topicDistribution
0,service_00026cb6595b4e93bb530c64bb4df92f,"[0.0005571014085451798, 0.003470435993264386, ..."
1,service_000726bc5df5449aa1b4555d54b754ff,"[0.00029594279024264676, 0.004474207121988655,..."
2,service_000b662279ce4ade9e3fd48f912c7aee,"[8.036530215281802e-05, 0.01300210717370562, 8..."
3,service_00199d7edaa14e9285d450f516538ba5,"[1.1849668326396843e-05, 4.465561080808407e-05..."
4,service_001bd5d853a8463b987ac5942a70436a,"[0.0003507513219250869, 0.00249677231122796, 3..."


In [4]:
vector_df = df.copy()

In [5]:
vector_df = vector_df.set_index('id')

In [6]:
entity_topic_df = pd.concat([df.drop(['topicDistribution'], axis=1), df['topicDistribution'].apply(pd.Series)], axis=1)

In [7]:
print(entity_topic_df.shape)
entity_topic_df.head()

(8847, 181)


,id,0,1,2,3,4,5,6,7,8,...,170,171,172,173,174,175,176,177,178,179
0,service_00026cb6595b4e93bb530c64bb4df92f,0.000557,0.003470,5.608440e-05,0.000703,0.000490,0.000795,0.000229,0.215924,0.000900,...,0.000490,0.002243,0.019150,0.001367,0.004295,0.048245,0.000629,0.000217,0.000786,0.001118
1,service_000726bc5df5449aa1b4555d54b754ff,0.000296,0.004474,2.979309e-05,0.001163,0.000260,0.000422,0.000121,0.000533,0.000478,...,0.000260,0.001981,0.011488,0.000726,0.006490,0.216087,0.000071,0.000115,0.000418,0.000068
2,service_000b662279ce4ade9e3fd48f912c7aee,0.000080,0.013002,8.090519e-06,0.000101,0.000071,0.002472,0.000033,0.000073,0.001344,...,0.000142,0.000324,0.004191,0.000554,0.000834,0.116544,0.000019,0.000031,0.000685,0.000018
3,service_00199d7edaa14e9285d450f516538ba5,0.000012,0.000045,6.315449e-07,0.000008,0.000006,0.000031,0.000003,0.000011,0.000016,...,0.000006,0.000042,0.000673,0.000049,0.000160,0.121060,0.000002,0.000002,0.000009,0.000001
4,service_001bd5d853a8463b987ac5942a70436a,0.000351,0.002497,3.531076e-05,0.000443,0.000308,0.000501,0.000144,0.000320,0.000566,...,0.000309,0.001412,0.024528,0.000861,0.003639,0.038170,0.000084,0.000137,0.000495,0.000080


In [8]:
entity_topic_df[entity_topic_df.id=='service_15496498543751ushi5opiop'].isna().sum().sum()

0

In [9]:
all_topics_id = entity_topic_df.columns.tolist()
all_topics_id.remove('id')
#all_topics_id

In [10]:
#topic to drop noncuisines and bad topic list
cuisine_topic_id = [0,3,6,7,11,14,15,16,17,18,19,20,21,22,23,25,28,29,30,31,32,33,34,35,36,37,39,44,45,46,47,49,52,55,56,59,60,61,63,64,65,66,67,69,70,71,72,73,76,77,78,79,82,83,86,87,89,90,91,93,94,95,97,98,99,101,104,105,107,108,110,112,114,116,117,119,120,124,125,127,128,130,132,133,134,135,136,137,138,139,140,141,142,143,145,147,148,150,154,159,160,161,162,163,164,165,166,169,170,171,173,174,176,177,179]
noncuisine_topic_id =   [2,5,8,13,27,40,54,58,74,80,100,102,103,109,111,113,115,118,123,129,151,152,153,155,167,168,172,175];

#bad_topic_id = [7,12,15,17,20,39,44,48,60,77,81,102,103,112,115,118,145,151,156,161,167,172,175]
#fine_dining_topic_id = [14,19,94]
#non_bad_topic_id = noncuisine_topic_id + bad_topic_id

In [11]:
len(cuisine_topic_id)+len(noncuisine_topic_id) 

143

In [12]:
#len(non_bad_topic_id)

# Clustering of Restaurants per topic

In [13]:
restaurant_topic = entity_topic_df.copy()
restaurant_topic['TopicID'] = restaurant_topic[restaurant_topic.columns[1:]].idxmax(axis=1)


In [14]:
restaurant_topic.head()

,id,0,1,2,3,4,5,6,7,8,...,171,172,173,174,175,176,177,178,179,TopicID
0,service_00026cb6595b4e93bb530c64bb4df92f,0.000557,0.003470,5.608440e-05,0.000703,0.000490,0.000795,0.000229,0.215924,0.000900,...,0.002243,0.019150,0.001367,0.004295,0.048245,0.000629,0.000217,0.000786,0.001118,7
1,service_000726bc5df5449aa1b4555d54b754ff,0.000296,0.004474,2.979309e-05,0.001163,0.000260,0.000422,0.000121,0.000533,0.000478,...,0.001981,0.011488,0.000726,0.006490,0.216087,0.000071,0.000115,0.000418,0.000068,54
2,service_000b662279ce4ade9e3fd48f912c7aee,0.000080,0.013002,8.090519e-06,0.000101,0.000071,0.002472,0.000033,0.000073,0.001344,...,0.000324,0.004191,0.000554,0.000834,0.116544,0.000019,0.000031,0.000685,0.000018,35
3,service_00199d7edaa14e9285d450f516538ba5,0.000012,0.000045,6.315449e-07,0.000008,0.000006,0.000031,0.000003,0.000011,0.000016,...,0.000042,0.000673,0.000049,0.000160,0.121060,0.000002,0.000002,0.000009,0.000001,77
4,service_001bd5d853a8463b987ac5942a70436a,0.000351,0.002497,3.531076e-05,0.000443,0.000308,0.000501,0.000144,0.000320,0.000566,...,0.001412,0.024528,0.000861,0.003639,0.038170,0.000084,0.000137,0.000495,0.000080,86


In [15]:
entity_name = pd.read_csv('entity_name.csv',header=0)
entity_name.columns = ['id','RestaurantName']
entity_name.head()

,id,RestaurantName
0,service_00026cb6595b4e93bb530c64bb4df92f,Lywa Oriental Foods
1,service_000726bc5df5449aa1b4555d54b754ff,Sprouts Restaurant & Cafe
2,service_000b662279ce4ade9e3fd48f912c7aee,Costa Coffee
3,service_00199d7edaa14e9285d450f516538ba5,Hayz
4,service_001bd5d853a8463b987ac5942a70436a,Qasr Al Jabal


In [16]:
all_rest_topic = pd.merge(entity_name,restaurant_topic[['id','TopicID']],how='inner',on='id')
all_rest_topic.head()

,id,RestaurantName,TopicID
0,service_00026cb6595b4e93bb530c64bb4df92f,Lywa Oriental Foods,7
1,service_000726bc5df5449aa1b4555d54b754ff,Sprouts Restaurant & Cafe,54
2,service_000b662279ce4ade9e3fd48f912c7aee,Costa Coffee,35
3,service_00199d7edaa14e9285d450f516538ba5,Hayz,77
4,service_001bd5d853a8463b987ac5942a70436a,Qasr Al Jabal,86


In [17]:
quick_look_df = pd.read_excel('zomato-dubai-quick-look-july.xlsx',header=0)
print(quick_look_df.shape)
quick_look_df.head()

(180, 6)


,TopicID,cuisine-1 non-cusine-0 bad-99,TopicName,Class,AlphaValue,Top30Words
0,0,1,Cuisine: English,English,0.01136,beer pub fish_chip belgian_beer pork mussel ir...
1,1,99,Bad,Location : Mall,0.07079,dubai_mall mall food_court court mall_emirate ...
2,2,0,Non Cuisine: Buffet,Buffet,0.00114,barbeque_nation bbq_nation singh kulfi nation ...
3,3,1,Cuisine: Sea Food,Sea Food,0.01434,crab lobster shrimp sea_food seafood paella sa...
4,4,99,Bad,Location: Mall,0.00999,very_nice شكرا azkadenya جدا sikka_cafe mahmou...


In [18]:
all_rest_name_topic_df = pd.merge(all_rest_topic,quick_look_df[['TopicID','TopicName']],how='inner',on='TopicID')
all_rest_name_topic_df.head()

,id,RestaurantName,TopicID,TopicName
0,service_00026cb6595b4e93bb530c64bb4df92f,Lywa Oriental Foods,7,Cuisine: Arabic
1,service_0d53c85a327a4ab7884b7337cba3384f,Al Saudi Al Yamni,7,Cuisine: Arabic
2,service_0e3d5156b19a43169a88d86f948ad19e,Arab Island Restaurant,7,Cuisine: Arabic
3,service_0ee6077f130340e5bc7d315c9159deb9,Bait Al Mandi,7,Cuisine: Arabic
4,service_0fa363aaaa654bc9b8e3b2fc11e0535f,Nablus,7,Cuisine: Arabic


In [19]:
all_rest_name_topic_df.to_csv("All Restaurants-Topic master.xlsx",header=True,index=False)

In [20]:
#quick_look_df[quick_look_df['CuisineModified'] == 0]['TopicID'].tolist()

In [21]:
quick_look_df['tupleTopics'] = list(zip(quick_look_df.TopicID, quick_look_df.TopicName))

In [22]:
quick_look_df.head()

,TopicID,cuisine-1 non-cusine-0 bad-99,TopicName,Class,AlphaValue,Top30Words,tupleTopics
0,0,1,Cuisine: English,English,0.01136,beer pub fish_chip belgian_beer pork mussel ir...,"(0, Cuisine: English)"
1,1,99,Bad,Location : Mall,0.07079,dubai_mall mall food_court court mall_emirate ...,"(1, Bad)"
2,2,0,Non Cuisine: Buffet,Buffet,0.00114,barbeque_nation bbq_nation singh kulfi nation ...,"(2, Non Cuisine: Buffet)"
3,3,1,Cuisine: Sea Food,Sea Food,0.01434,crab lobster shrimp sea_food seafood paella sa...,"(3, Cuisine: Sea Food)"
4,4,99,Bad,Location: Mall,0.00999,very_nice شكرا azkadenya جدا sikka_cafe mahmou...,"(4, Bad)"


import re
quick_look_df.TopicName = quick_look_df['TopicName'].str.replace('[^a-zA-Z0-9]',' ')
quick_look_df.TopicName = quick_look_df['TopicName'].apply(lambda x : re.sub(' +', ' ', str(x)))

In [23]:
topic_dictionary = {}
for item in range(quick_look_df.shape[0]):
    topic_dictionary[item] = quick_look_df.loc[item,'tupleTopics']

In [24]:
#topic_dictionary

In [25]:
#create a dictionary of topic id and tuple of topic id ,topic name {1:(1,'cuisine - roast')}

In [26]:
type(str(topic_dictionary[all_topics_id[2]]))

str

In [27]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('per_topic_top_100_restaurants.xlsx', engine='xlsxwriter')
for column in all_topics_id:
    temp_df = entity_topic_df.sort_values(by=[column],ascending=False).reset_index(drop=True).loc[:,['id',column]]
    temp_df.rename(columns = {column:topic_dictionary[column]},inplace=True)
    result_df = pd.merge(temp_df,all_rest_name_topic_df,on='id',how='inner')
    result_df.to_excel(writer,sheet_name=str(column))
    
# Close the Pandas Excel writer and output the Excel file.
writer.save()

    

KeyboardInterrupt: 

# Topic Analysis

### On all topics

In [28]:
entity_topic_df.head()

,id,0,1,2,3,4,5,6,7,8,...,170,171,172,173,174,175,176,177,178,179
0,service_00026cb6595b4e93bb530c64bb4df92f,0.000557,0.003470,5.608440e-05,0.000703,0.000490,0.000795,0.000229,0.215924,0.000900,...,0.000490,0.002243,0.019150,0.001367,0.004295,0.048245,0.000629,0.000217,0.000786,0.001118
1,service_000726bc5df5449aa1b4555d54b754ff,0.000296,0.004474,2.979309e-05,0.001163,0.000260,0.000422,0.000121,0.000533,0.000478,...,0.000260,0.001981,0.011488,0.000726,0.006490,0.216087,0.000071,0.000115,0.000418,0.000068
2,service_000b662279ce4ade9e3fd48f912c7aee,0.000080,0.013002,8.090519e-06,0.000101,0.000071,0.002472,0.000033,0.000073,0.001344,...,0.000142,0.000324,0.004191,0.000554,0.000834,0.116544,0.000019,0.000031,0.000685,0.000018
3,service_00199d7edaa14e9285d450f516538ba5,0.000012,0.000045,6.315449e-07,0.000008,0.000006,0.000031,0.000003,0.000011,0.000016,...,0.000006,0.000042,0.000673,0.000049,0.000160,0.121060,0.000002,0.000002,0.000009,0.000001
4,service_001bd5d853a8463b987ac5942a70436a,0.000351,0.002497,3.531076e-05,0.000443,0.000308,0.000501,0.000144,0.000320,0.000566,...,0.000309,0.001412,0.024528,0.000861,0.003639,0.038170,0.000084,0.000137,0.000495,0.000080


In [29]:
df = entity_topic_df.copy()

In [30]:
merged_df = pd.merge(df,all_rest_name_topic_df,on='id',how='inner')
merged_df.head()

,id,0,1,2,3,4,5,6,7,8,...,173,174,175,176,177,178,179,RestaurantName,TopicID,TopicName
0,service_00026cb6595b4e93bb530c64bb4df92f,0.000557,0.003470,5.608440e-05,0.000703,0.000490,0.000795,0.000229,0.215924,0.000900,...,0.001367,0.004295,0.048245,0.000629,0.000217,0.000786,0.001118,Lywa Oriental Foods,7,Cuisine: Arabic
1,service_000726bc5df5449aa1b4555d54b754ff,0.000296,0.004474,2.979309e-05,0.001163,0.000260,0.000422,0.000121,0.000533,0.000478,...,0.000726,0.006490,0.216087,0.000071,0.000115,0.000418,0.000068,Sprouts Restaurant & Cafe,54,Non Cuisine: Service Good
2,service_000b662279ce4ade9e3fd48f912c7aee,0.000080,0.013002,8.090519e-06,0.000101,0.000071,0.002472,0.000033,0.000073,0.001344,...,0.000554,0.000834,0.116544,0.000019,0.000031,0.000685,0.000018,Costa Coffee,35,Cuisine: Cafe startbucks
3,service_00199d7edaa14e9285d450f516538ba5,0.000012,0.000045,6.315449e-07,0.000008,0.000006,0.000031,0.000003,0.000011,0.000016,...,0.000049,0.000160,0.121060,0.000002,0.000002,0.000009,0.000001,Hayz,77,Cuisine: Healthy
4,service_001bd5d853a8463b987ac5942a70436a,0.000351,0.002497,3.531076e-05,0.000443,0.000308,0.000501,0.000144,0.000320,0.000566,...,0.000861,0.003639,0.038170,0.000084,0.000137,0.000495,0.000080,Qasr Al Jabal,86,Cuisine: Fast Food


In [31]:
merged_df.shape

(8334, 184)

In [32]:
merged_df[merged_df.RestaurantName == 'Couqley French Bistro & Bar']

,id,0,1,2,3,4,5,6,7,8,...,173,174,175,176,177,178,179,RestaurantName,TopicID,TopicName
2308,service_5428ca23c9504273b629c0bddf988431,0.000497,0.000322,1.908095e-07,0.000021,0.000002,0.000009,7.774729e-07,0.000002,0.000006,...,0.000018,0.080698,0.036598,4.559646e-07,7.382203e-07,0.000004,4.331702e-07,Couqley French Bistro & Bar,108,Cuisine: French Cafe


In [33]:
merged_df[merged_df.RestaurantName == 'Chamas - Crowne Plaza Dubai']

,id,0,1,2,3,4,5,6,7,8,...,173,174,175,176,177,178,179,RestaurantName,TopicID,TopicName
6699,service_4ede3c3a674b4080b7bf5401dac588db,0.000002,0.00005,2.341629e-07,0.000009,0.000004,0.000007,9.541207e-07,0.000006,0.032666,...,0.000031,0.039007,0.06972,5.595633e-07,0.28439,0.000003,5.315899e-07,Chamas - Crowne Plaza Dubai,177,Cuisine: Steak?


In [34]:
merged_df[merged_df.RestaurantName == 'Frevo - Fairmont The Palm']

,id,0,1,2,3,4,5,6,7,8,...,173,174,175,176,177,178,179,RestaurantName,TopicID,TopicName
5875,service_a97de12289b8450991972f361eeaa90f,0.000004,0.000036,4.017629e-07,0.000019,0.000004,0.000034,0.000002,0.000014,0.012681,...,0.000024,0.031954,0.05293,9.600659e-07,0.28787,0.000006,0.000004,Frevo - Fairmont The Palm,177,Cuisine: Steak?


### only cuisine related topics

In [35]:
#only cuisine related topics
only_cuisine_df = merged_df.loc[:,['id'] + cuisine_topic_id]

In [36]:
only_cuisine_df.head()

,id,0,3,6,7,11,14,15,16,17,...,165,166,169,170,171,173,174,176,177,179
0,service_00026cb6595b4e93bb530c64bb4df92f,0.000557,0.000703,0.000229,0.215924,0.000238,0.000243,0.000097,0.001428,0.000328,...,0.000177,0.000379,0.000916,0.000490,0.002243,0.001367,0.004295,0.000629,0.000217,0.001118
1,service_000726bc5df5449aa1b4555d54b754ff,0.000296,0.001163,0.000121,0.000533,0.000127,0.000129,0.000051,0.000759,0.000174,...,0.000094,0.000201,0.000487,0.000260,0.001981,0.000726,0.006490,0.000071,0.000115,0.000068
2,service_000b662279ce4ade9e3fd48f912c7aee,0.000080,0.000101,0.000033,0.000073,0.000034,0.000035,0.000014,0.031924,0.000047,...,0.000026,0.000055,0.010348,0.000142,0.000324,0.000554,0.000834,0.000019,0.000031,0.000018
3,service_00199d7edaa14e9285d450f516538ba5,0.000012,0.000008,0.000003,0.000011,0.000003,0.000008,0.000001,0.000016,0.012857,...,0.000002,0.000010,0.000033,0.000006,0.000042,0.000049,0.000160,0.000002,0.000002,0.000001
4,service_001bd5d853a8463b987ac5942a70436a,0.000351,0.000443,0.000144,0.000320,0.000150,0.000153,0.000061,0.000899,0.000207,...,0.000111,0.000238,0.000577,0.000309,0.001412,0.000861,0.003639,0.000084,0.000137,0.000080


In [37]:
only_cuisine_df['TopicID'] = only_cuisine_df[only_cuisine_df.columns[1:]].idxmax(axis=1)

In [38]:
only_cuisine_df.head(3)

,id,0,3,6,7,11,14,15,16,17,...,166,169,170,171,173,174,176,177,179,TopicID
0,service_00026cb6595b4e93bb530c64bb4df92f,0.000557,0.000703,0.000229,0.215924,0.000238,0.000243,0.000097,0.001428,0.000328,...,0.000379,0.000916,0.000490,0.002243,0.001367,0.004295,0.000629,0.000217,0.001118,7
1,service_000726bc5df5449aa1b4555d54b754ff,0.000296,0.001163,0.000121,0.000533,0.000127,0.000129,0.000051,0.000759,0.000174,...,0.000201,0.000487,0.000260,0.001981,0.000726,0.006490,0.000071,0.000115,0.000068,18
2,service_000b662279ce4ade9e3fd48f912c7aee,0.000080,0.000101,0.000033,0.000073,0.000034,0.000035,0.000014,0.031924,0.000047,...,0.000055,0.010348,0.000142,0.000324,0.000554,0.000834,0.000019,0.000031,0.000018,35


In [39]:
only_cuisine_df = pd.merge(only_cuisine_df,quick_look_df[['TopicID','TopicName']],on='TopicID',how='inner')
only_cuisine_df = pd.merge(only_cuisine_df,all_rest_name_topic_df[['id','RestaurantName']],on='id',how='inner')

In [40]:
only_cuisine_df.head(3)

,id,0,3,6,7,11,14,15,16,17,...,170,171,173,174,176,177,179,TopicID,TopicName,RestaurantName
0,service_00026cb6595b4e93bb530c64bb4df92f,0.000557,0.000703,0.000229,0.215924,0.000238,0.000243,0.000097,0.001428,0.000328,...,0.000490,0.002243,0.001367,0.004295,0.000629,0.000217,0.001118,7,Cuisine: Arabic,Lywa Oriental Foods
1,service_62b846a26e224c7ab36c0e6115bbef23,0.000075,0.000095,0.000031,0.314980,0.000032,0.000033,0.000013,0.000192,0.000044,...,0.000133,0.000436,0.000251,0.000845,0.000018,0.000029,0.000017,7,Cuisine: Arabic,Tibba Restaurant for Mandi and Mazbi
2,service_3fc1be201ab84503a0bea64b8c42182c,0.000090,0.000113,0.000037,0.236789,0.000038,0.000039,0.000016,0.000310,0.000053,...,0.000559,0.000522,0.000460,0.001332,0.000022,0.000035,0.000021,7,Cuisine: Arabic,Sana'a


In [41]:
only_cuisine_df[only_cuisine_df.RestaurantName=='Couqley French Bistro & Bar']

,id,0,3,6,7,11,14,15,16,17,...,170,171,173,174,176,177,179,TopicID,TopicName,RestaurantName
7233,service_5428ca23c9504273b629c0bddf988431,0.000497,0.000021,7.774729e-07,0.000002,8.113812e-07,8.250752e-07,3.298137e-07,0.00001,0.000001,...,0.000002,0.000008,0.000018,0.080698,4.559646e-07,7.382203e-07,4.331702e-07,108,Cuisine: French Cafe,Couqley French Bistro & Bar


In [42]:
only_cuisine_df[only_cuisine_df.RestaurantName=='Chamas - Crowne Plaza Dubai']

,id,0,3,6,7,11,14,15,16,17,...,170,171,173,174,176,177,179,TopicID,TopicName,RestaurantName
8134,service_4ede3c3a674b4080b7bf5401dac588db,0.000002,0.000009,9.541207e-07,0.000006,9.957333e-07,0.000001,4.047499e-07,0.000018,0.000003,...,0.000058,0.000011,0.000031,0.039007,5.595633e-07,0.28439,5.315899e-07,177,Cuisine: Steak?,Chamas - Crowne Plaza Dubai


In [43]:
only_cuisine_df[only_cuisine_df.RestaurantName=='Catch 22']

,id,0,3,6,7,11,14,15,16,17,...,170,171,173,174,176,177,179,TopicID,TopicName,RestaurantName
7823,service_2c9cc58d0798443eba96dd41531b358a,0.000009,0.016178,1.367180e-06,3.037535e-06,4.389504e-06,1.450888e-06,5.799747e-07,0.000009,1.964405e-06,...,0.000003,0.000019,0.015432,0.000390,8.018101e-07,0.000004,7.617263e-07,29,"Cuisine: Indian,Maharastra",Catch 22
7826,service_6deae7241061423cb403881da14cb21e,0.000002,0.033139,3.977048e-07,8.836018e-07,4.150502e-07,4.220551e-07,1.687114e-07,0.000005,5.714345e-07,...,0.000003,0.000004,0.034514,0.002807,2.332420e-07,0.000003,2.215819e-07,29,"Cuisine: Indian,Maharastra",Catch 22


### Cuisine and non cuisine(only fine dining) related topics

#only cuisine related topics
cuisine_Non_cusine_df = merged_df.loc[:,['id'] + cuisine_topic_id+fine_dining_topic_id]
cuisine_Non_cusine_df.head()

cuisine_Non_cusine_df['TopicID'] = cuisine_Non_cusine_df[cuisine_Non_cusine_df.columns[1:]].idxmax(axis=1)

cuisine_Non_cusine_df = pd.merge(cuisine_Non_cusine_df,quick_look_df[['TopicID','TopicName']],on='TopicID',how='inner')
cuisine_Non_cusine_df = pd.merge(cuisine_Non_cusine_df,all_rest_name_topic_df[['id','RestaurantName']],on='id',how='inner')

cuisine_Non_cusine_df[cuisine_Non_cusine_df['RestaurantName']=='Benares Restaurant & Bar']

cuisine_Non_cusine_df[cuisine_Non_cusine_df.RestaurantName=='Al Arez 2']

cuisine_Non_cusine_df[cuisine_Non_cusine_df.RestaurantName=='Tayyabs']

cuisine_Non_cusine_df[cuisine_Non_cusine_df.RestaurantName=='Rossopomodoro - Covent Garden']

## Top 10, topic contributors to a given restaurant (cusine and fine dining topics)

In [44]:
#merged_df has all topics in the column

In [44]:
restaurant_name = 'Catch 22'
per_rest_df = merged_df[merged_df.RestaurantName == restaurant_name].loc[:,cuisine_topic_id+noncuisine_topic_id].reset_index(drop=True).T.sort_values(by=0,ascending=False).T.iloc[:,:10]
print("ogignally==>",restaurant_name +' belongs to:')
print("Topic Id :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicID'].values[0])
print("Cuisine Name :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicName'].values[0])
rest_cp = per_rest_df.copy()
for item in per_rest_df.columns:
    rest_cp.rename(columns={item:topic_dictionary[item]},inplace=True)
rest_cp

ogignally==> Catch 22 belongs to:
Topic Id : 74
Cuisine Name : Non Cuisine: Service Good


,"(74, Non Cuisine: Service Good)","(54, Non Cuisine: Service Good)","(29, Cuisine: Indian,Maharastra)","(152, Non Cuisine: Service Good)","(58, Non Cuisine: Ambience)","(155, Non Cuisine: Good Taste)","(80, Non Cuisine: Good Taste)","(175, Non Cuisine: Service Bad)","(3, Cuisine: Sea Food)","(173, Cuisine: American)"
0,0.252054,0.206208,0.201825,0.043440,0.039467,0.030062,0.023392,0.018823,0.016178,0.015432
1,0.182134,0.113249,0.174629,0.033419,0.034764,0.043796,0.078050,0.026522,0.033139,0.034514


In [45]:
restaurant_name = 'Joe\'s Crab Shack'
per_rest_df = merged_df[merged_df.RestaurantName == restaurant_name].loc[:,cuisine_topic_id+noncuisine_topic_id].reset_index(drop=True).T.sort_values(by=0,ascending=False).T.iloc[:,:10]
print("ogignally==>",restaurant_name +' belongs to:')
print("Topic Id :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicID'].values[0])
print("Cuisine Name :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicName'].values[0])
rest_cp = per_rest_df.copy()
for item in per_rest_df.columns:
    rest_cp.rename(columns={item:topic_dictionary[item]},inplace=True)
rest_cp

ogignally==> Joe's Crab Shack belongs to:
Topic Id : 119
Cuisine Name : Cuisine: Indian


,"(119, Cuisine: Indian)","(74, Non Cuisine: Service Good)","(3, Cuisine: Sea Food)","(54, Non Cuisine: Service Good)","(152, Non Cuisine: Service Good)","(155, Non Cuisine: Good Taste)","(80, Non Cuisine: Good Taste)","(13, Non Cuisine: Ambience)","(168, Non Cuisine: Service Good)","(129, Non Cuisine: Good Taste)"
0,0.172088,0.168752,0.128328,0.124212,0.058523,0.058280,0.052385,0.029094,0.026996,0.023840
1,0.228827,0.066903,0.163034,0.052406,0.029814,0.092082,0.064976,0.054729,0.029548,0.013417


In [46]:
restaurant_name = 'The Cheesecake Factory'
per_rest_df = merged_df[merged_df.RestaurantName == restaurant_name].loc[:,cuisine_topic_id+noncuisine_topic_id].reset_index(drop=True).T.sort_values(by=0,ascending=False).T.iloc[:,:20]
print("ogignally==>",restaurant_name +' belongs to:')
print("Topic Id :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicID'].values[0])
print("Cuisine Name :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicName'].values[0])
rest_cp = per_rest_df.copy()
for item in per_rest_df.columns:
    rest_cp.rename(columns={item:topic_dictionary[item]},inplace=True)
rest_cp

ogignally==> The Cheesecake Factory belongs to:
Topic Id : 70
Cuisine Name : Cuisine: Dessert Cake


,"(70, Cuisine: Dessert Cake)","(155, Non Cuisine: Good Taste)","(13, Non Cuisine: Ambience)","(152, Non Cuisine: Service Good)","(80, Non Cuisine: Good Taste)","(173, Cuisine: American)","(54, Non Cuisine: Service Good)","(102, Non Cuisine: Ambience)","(168, Non Cuisine: Service Good)","(175, Non Cuisine: Service Bad)","(103, Non Cuisine: Service Good)","(118, Non Cuisine: Ambience)","(95, Cuisine: Pasta Italian)","(74, Non Cuisine: Service Good)","(105, Cuisine:? (chicken))","(124, Cuisine: Break Fast)","(145, Cuisine: Fast Food)","(78, Cuisine: Dessert Cake)","(129, Non Cuisine: Good Taste)","(123, Non Cuisine: Ambience)"
0,0.254907,0.098708,0.094741,0.070858,0.052165,0.051207,0.049785,0.044473,0.043451,0.035104,0.033875,0.019439,0.017035,0.015725,0.014845,0.011800,0.008925,0.008331,0.006919,0.006541
1,0.305024,0.109486,0.118593,0.063704,0.051373,0.052869,0.034127,0.031760,0.041516,0.022967,0.022559,0.001600,0.021098,0.007405,0.007591,0.004137,0.012484,0.011209,0.008675,0.021411
2,0.281418,0.110424,0.103256,0.059050,0.067502,0.059334,0.038344,0.035395,0.045497,0.032862,0.031956,0.001945,0.019263,0.010846,0.014071,0.006089,0.012827,0.009030,0.002766,0.012101
3,0.276172,0.141887,0.057698,0.061172,0.051076,0.053798,0.035860,0.014834,0.038505,0.033016,0.041841,0.015895,0.035114,0.020610,0.015262,0.007858,0.007301,0.006939,0.006916,0.018745


### outPut analysis

### For 1st Input - 'Tayyabs'

In [ ]:
restaurant_name = "Preto Camden"
per_rest_df = merged_df[merged_df.RestaurantName == restaurant_name].loc[:,cuisine_topic_id+noncuisine_topic_id].reset_index(drop=True).T.sort_values(by=0,ascending=False).T.iloc[:,:10]
print("ogignally==>",restaurant_name +' belongs to:')
print("Topic Id :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicID'].values[0])
print("Cuisine Name :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicName'].values[0])
rest_cp = per_rest_df.copy()
for item in per_rest_df.columns:
    rest_cp.rename(columns={item:topic_dictionary[item]},inplace=True)
rest_cp

In [ ]:
restaurant_name = 'Abeno Too'
per_rest_df = merged_df[merged_df.RestaurantName == restaurant_name].loc[:,cuisine_topic_id].reset_index(drop=True).T.sort_values(by=0,ascending=False).T.iloc[:,:10]
print("ogignally==>",restaurant_name +' belongs to:')
print("Topic Id :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicID'].values[0])
print("Cuisine Name :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicName'].values[0])
rest_cp = per_rest_df.copy()
for item in per_rest_df.columns:
    rest_cp.rename(columns={item:topic_dictionary[item]},inplace=True)
rest_cp

### For 2nd Input 'Al Arez 2'

In [ ]:
restaurant_name = 'Black Roe'
per_rest_df = merged_df[merged_df.RestaurantName == restaurant_name].loc[:,cuisine_topic_id+noncuisine_topic_id].reset_index(drop=True).T.sort_values(by=0,ascending=False).T.iloc[:,:10]
print("ogignally==>",restaurant_name +' belongs to:')
print("Topic Id :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicID'].values[0])
print("Cuisine Name :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicName'].values[0])
rest_cp = per_rest_df.copy()
for item in per_rest_df.columns:
    rest_cp.rename(columns={item:topic_dictionary[item]},inplace=True)
rest_cp

In [ ]:
restaurant_name = 'SUSHISAMBA City'
per_rest_df = merged_df[merged_df.RestaurantName == restaurant_name].loc[:,cuisine_topic_id+noncuisine_topic_id].reset_index(drop=True).T.sort_values(by=0,ascending=False).T.iloc[:,:10]
print("ogignally==>",restaurant_name +' belongs to:')
print("Topic Id :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicID'].values[0])
print("Cuisine Name :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicName'].values[0])
rest_cp = per_rest_df.copy()
for item in per_rest_df.columns:
    rest_cp.rename(columns={item:topic_dictionary[item]},inplace=True)
rest_cp

### For 3rd Input ''Rossopomodoro - Covent Garden''

In [ ]:
restaurant_name = 'Dishoom King\'s Cross'
per_rest_df = merged_df[merged_df.RestaurantName == restaurant_name].loc[:,cuisine_topic_id+noncuisine_topic_id].reset_index(drop=True).T.sort_values(by=0,ascending=False).T.iloc[:,:10]
print("ogignally==>",restaurant_name +' belongs to:')
print("Topic Id :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicID'].values[0])
print("Cuisine Name :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicName'].values[0])
rest_cp = per_rest_df.copy()
for item in per_rest_df.columns:
    rest_cp.rename(columns={item:topic_dictionary[item]},inplace=True)
rest_cp

In [ ]:
restaurant_name = 'Fiddie’s Italian Kitchen'
per_rest_df = merged_df[merged_df.RestaurantName == restaurant_name].loc[:,cuisine_topic_id+noncuisine_topic_id].reset_index(drop=True).T.sort_values(by=0,ascending=False).T.iloc[:,:10]
print("ogignally==>",restaurant_name +' belongs to:')
print("Topic Id :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicID'].values[0])
print("Cuisine Name :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicName'].values[0])
rest_cp = per_rest_df.copy()
for item in per_rest_df.columns:
    rest_cp.rename(columns={item:topic_dictionary[item]},inplace=True)
rest_cp

In [ ]:
restaurant_name = 'Marcus'
per_rest_df = merged_df[merged_df.RestaurantName == restaurant_name].loc[:,cuisine_topic_id+noncuisine_topic_id].reset_index(drop=True).T.sort_values(by=0,ascending=False).T.iloc[:,:10]
print("ogignally==>",restaurant_name +' belongs to:')
print("Topic Id :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicID'].values[0])
print("Cuisine Name :",merged_df[merged_df.RestaurantName == restaurant_name]['TopicName'].values[0])
rest_cp = per_rest_df.copy()
for item in per_rest_df.columns:
    rest_cp.rename(columns={item:topic_dictionary[item]},inplace=True)
rest_cp

In [ ]:
quick_look_df.head()

In [ ]:
sushi = [{"id":"service_15496504974131ltuuxseu3q","name":"SUSHISAMBA City","topicDistributions":[2.773623708289316E-8,5.963639807568622E-7,1.2225988817053029E-7,6.61356005164357E-7,2.138889428252942E-7,1.1366669980477171E-7,3.3459335221990844E-8,3.0412014921276225E-7,1.5085865323638622E-7,4.189261216671244E-8,8.274557111884182E-8,0.1693214131351682,4.931678855678753E-8,4.053440734921401E-8,0.0011622917665240356,9.34739386901643E-9,6.429871662192856E-8,3.730901654450294E-8,1.3464418114947702E-4,9.797299717151547E-9,4.504626196526025E-8,0.022302619946104656,7.794316744474355E-8,4.8988313056941187E-8,0.0435273948090718,1.8054786989466496E-7,1.744209029161792E-7,7.947232336457585E-6,2.9135402532338007E-8,0.06992387666621014,5.015469795101532E-7,9.313277274732739E-8,1.0213321983042516E-7,7.113751600385322E-7,0.04163598011051877,1.7820221106004689E-6,3.9654611847035855E-7,1.4268595455932093E-7,7.317931147969939E-8,1.502275529148647E-4,4.6127309904747743E-8,8.068886941542148E-6,4.042197107337008E-8,3.079589270517878E-8,3.874823314881955E-4,0.08544494854665607,9.648907562167585E-8,1.4841717540960652E-7,7.665199591523887E-8,1.1076803172380406E-7,2.919030709653466E-8,5.139376114128827E-4,3.1137057382653547E-7,6.507734332295095E-8,2.5524458976353062E-8,1.1683648970949462E-7,1.8556206280638195E-7,4.9912895048211184E-8,0.11686870711157228,3.9820404774264023E-7,7.559394088811147E-8,9.408457558826296E-4,0.0012254713520492154,0.1020152292109129,7.621301948403495E-8,0.0012655443180557193,1.349602023500418E-7,3.221359677479706E-8,2.402523870596377E-5,8.947944706165909E-8,0.005732039152493634,1.6200933305209183E-5,5.766450591385912E-8,3.145255195895205E-8,1.2444705134136084E-7,6.843523462181989E-8,0.003399611388833922,6.717209859185379E-8,8.182081040392266E-8,8.623927895683825E-4,0.0010911802073968666,2.7519621598338892E-8,1.5957853832490113E-5,2.938749058005544E-8,2.1397489360965546E-8,1.934253696833725E-7,0.05656450109072204,1.1523224862711293E-7,0.005415972424942595,1.2234625513391054E-7,6.426096443133419E-7,0.010127072295634736,7.966269283838715E-6,9.044021325198396E-8,6.979494268763597E-9,6.52024169604622E-8,7.853247995493043E-8,0.008094635428781208,1.5695169505377764E-7,4.669549815985245E-8,0.0010119614163058694,0.0010361549072435308,4.840190004247609E-8,9.411803616855447E-8,3.869296119104255E-8,2.8583620751424984E-8,2.1329406890763884E-8,1.8010645304774522E-7,4.058334732386099E-5,3.382381355813538E-8,3.053987179184017E-8,1.203233388932913E-7,1.3877428032294028E-8,7.259584240911694E-4,5.764267969547571E-7,1.9156204227793982E-8,1.0211929809170336E-7,4.6063099989287134E-7,7.859213897035023E-8,6.197185441602208E-8,0.0597692145515575,1.0771614083225585E-7,3.167311516635546E-5,4.720288164740226E-7,5.910162058184263E-8,2.5020839648342015E-7,3.5881439093523336E-8,0.0845947308405022,1.5467127769619853E-7,0.008696167766785673,1.345547118686193E-7,1.7207479109578947E-7,8.321160379999147E-7,1.1012192992840976E-6,1.440358689875827E-7,8.271502468459504E-9,1.0633715015262738E-7,9.914953486162198E-8,1.2343660952590806E-8,8.047049461467864E-8,1.217676572709323E-7,3.312906032093949E-8,1.9731000391283128E-8,6.404946593919785E-5,3.818834839525942E-8,3.68290812854367E-8,1.919470698792991E-8,3.07429875135166E-7,7.795223537088913E-8,1.1952153240972225E-7,0.0022847572213190765,4.8635863340659286E-8,4.745835277516828E-8,9.407990338017603E-8,9.82639395467634E-8,8.516158163528794E-8,1.584068838113727E-5,3.9736527486280123E-7,0.006325056755133959,8.223993966703502E-9,5.3353952175705736E-8,8.81300158753443E-8,0.02935442105666405,6.782255841431178E-8,1.8382176857989178E-8,6.049377418938795E-8,3.3675783872316745E-7,7.603519148054634E-8,3.95618023264405E-4,3.044294122227484E-8,2.6892062096080114E-4,4.506475679678031E-4,6.32496914714037E-4,7.395686887040051E-8,0.008523878831115278,0.04552672427456352,2.609460336809748E-7,1.0847684044301366E-7,0.0020434085888100116,5.824100474086898E-8],"ThaKrowdScore":3.3509118725,"CuisineList":["[\"Japanese\"","\"Sushi\"","\"\"","\"SouthAmerican\"","\"VegetarianFriendly\"","\"VeganOptions\"","\"Options\"]"],"priceRange":"NaN","latlong":""},
{"id":"service_15496531771085gek7lczh2n","name":"SUSHISAMBA Covent Garden","topicDistributions":[0.004868352183928464,3.337562698689796E-5,6.842298587283559E-6,3.701291836220032E-5,1.1970336577502669E-5,6.361379117332138E-6,1.8725582490436035E-6,0.02921781966364187,8.442834075547876E-6,2.3445282449964412E-6,4.630872094212279E-6,0.04955704073926565,2.760023730766068E-6,2.2685160463671667E-6,9.74197467511178E-6,5.231287286599251E-7,3.5984902692918877E-6,2.0880064182565184E-6,1.3969911274935981E-5,5.483077975693595E-7,2.521022873645971E-6,0.0018272776013353509,4.362104631992659E-6,2.7416405350819614E-6,0.018546924599786217,1.0104396900757967E-5,9.761499993779352E-6,2.332057892134597E-6,0.004425994132134364,0.030481989476406388,2.8069174941269474E-5,5.212193867787627E-6,5.715905651620622E-6,3.981225016148445E-5,0.11488694540048287,9.973121644657863E-5,2.2192781187710328E-5,7.985447393381278E-6,4.095494499860206E-6,1.2236515587758725E-6,2.5815239333134213E-6,9.14048169296121E-6,2.2622235281677938E-6,1.723498166928406E-6,6.143112256996299E-6,0.07751750345726845,5.400030015515968E-6,8.30619629078434E-6,4.289843964455094E-6,6.199154590447698E-6,1.6336412538706647E-6,4.255918971017615E-6,1.7425915149237894E-5,3.6420662651164198E-6,1.4284813458317448E-6,6.538777030184958E-6,1.0385017189141957E-5,2.7933849473120016E-6,0.09560968026952961,2.2285567524155577E-5,4.2306297076312675E-6,4.690594867075655E-6,6.109002044309373E-6,0.07354797614421861,4.2652765625048485E-6,0.008000473026001556,7.5530741578232626E-6,1.8028402528560903E-6,1.7268120674593834E-5,5.007734780248887E-6,2.833548358137048E-5,2.1815692815545073E-5,3.2272053676383743E-6,1.760248230679098E-6,6.964703602516837E-6,3.829991309334159E-6,0.008861011115292663,3.7592996540193026E-6,4.5791176796782085E-6,3.841853291863285E-5,1.1896847476873011E-5,1.5401410127437025E-6,8.211702515280691E-6,1.6446766661462593E-6,1.1975146829983764E-6,1.0825088698619898E-5,0.04071539727611236,6.448995363802834E-6,0.019946689462177095,6.847132131304967E-6,3.5963774605511306E-5,2.6120130373718533E-6,3.3974643808957505E-6,5.06150424826607E-6,3.9060876375499566E-7,3.6490660357366583E-6,4.3950856220480075E-6,0.01943022795164385,8.783832354241264E-6,4.45049679062814E-4,2.7652519031953247E-6,2.944964167861518E-5,2.708821815872578E-6,5.267334328130279E-6,2.1654591514594845E-6,1.5996879337410235E-6,1.193704434218909E-6,1.0079692920464206E-5,5.9073199821076266E-5,1.892956350512654E-6,1.7091699063691583E-6,6.733919227690366E-6,7.766529778601778E-7,0.00125156967420403,3.225983859051664E-5,1.072080721542358E-6,5.715126518786344E-6,2.5779304128185155E-5,4.39842445053079E-6,3.468266970199136E-6,0.09928268849993128,6.028354918958899E-6,2.8466683308632057E-6,2.6417185165525468E-5,3.307633771505328E-6,1.4002961915007326E-5,2.008111766607147E-6,0.07417853864499224,8.65620835018184E-6,2.740300676106441E-6,7.530380803611093E-6,9.630199386241817E-6,4.656953704489973E-5,6.162995376922968E-5,8.060995618753168E-6,4.6291625570368335E-7,5.951179435484398E-6,5.548923137957059E-6,6.908154028409903E-7,4.503546992060337E-6,6.814750788579222E-6,1.8540743495186275E-6,1.1042493014115957E-6,4.505005511448291E-5,2.137218397510027E-6,2.0611467474827634E-6,1.0742355360545152E-6,1.7205373175150217E-5,4.362612120255441E-6,6.68904597079562E-6,2.7783948688236468E-6,2.7219156176794487E-6,2.656016008254543E-6,5.265200219159992E-6,5.499360622708313E-6,4.766084595964969E-6,1.6545150822953787E-6,2.2238625435653344E-5,0.09648508072360591,4.6025743309789427E-7,2.9859643834176304E-6,4.932213599607209E-6,0.019969368654946595,0.002215977484577974,1.0287621281902432E-6,3.3855459208187863E-6,1.884671840152722E-5,4.255324383460219E-6,1.902233960650878E-5,1.7037451680586147E-6,0.002219540888346922,1.7070924803408562E-6,2.905203805681895E-6,4.139010651523173E-6,0.012924810445862633,0.09106295400840492,1.460389588384193E-5,6.070927621665573E-6,0.001096054665922635,3.2594692374050325E-6],"ThaKrowdScore":1.8711614865,"CuisineList":["[]"],"priceRange":0.0,"latlong":""}]


In [ ]:
my_t = pd.DataFrame(sushi)

In [ ]:
my_t  = pd.concat([my_t.drop(['topicDistributions'], axis=1), my_t['topicDistributions'].apply(pd.Series)], axis=1)

In [ ]:
my_t.drop(columns=['CuisineList','latlong','priceRange'],inplace=True)

In [ ]:
my_t[]

In [ ]:
topicname = quick_look_df.TopicName.tolist()
col = ['ThaKrowdScore','id','name'] + topicname

In [ ]:
my_t.columns = col

In [ ]:
my_t

In [ ]:
my_t.to_csv('Sushibothoutlets.csv',index=False,header=True)

weight for each topics

In [ ]:
topicweight = [2.5,2.5,1,2.5,2.5,1,2.5,1,2.5,1,2.5,1,1,2.5,1,1,2.5,1,2.5,1,1,1,1,1,1,2.5,2.5,2.5,2.5,1,2.5,1,1,1,1,2.5,1,1,2.5,1,2.5,1,2.5,1,1,1,1,2.5,1,1,1,2.5,2.5,1,2.5,1,2.5,1,1,2.5,1,2.5,2.5,1,1,1,2.5,2.5,2.5,1,1,2.5,1,1,2.5,2.5,1,1,1,1,1,1,1,2.5,2.5,1,2.5,1,1,1,2.5,2.5,2.5,2.5,1,1,2.5,1,1,1,1,2.5,1,1,1,1,2.5,2.5,2.5,1,1,2.5,1,1,2.5,1,1,1,1,2.5,1,1,2.5,2.5,2.5,2.5,2.5,1,2.5,2.5,1,2.5,2.5,2.5,2.5,2.5,2.5,1,1,2.5,2.5,1,2.5,2.5,2.5,1,2.5,2.5,2.5,2.5,2.5,1,1,1,2.5,1,1,2.5,1,1,2.5,1,1,2.5,2.5,2.5,2.5,1,1,2.5,2.5,2.5,1,2.5,1,1,2.5,1,1,1]

In [ ]:
len(topicweight)

In [ ]:
import collections

In [ ]:
counter=collections.Counter(topicweight)

In [ ]:
print(counter)

In [ ]:
 topicperweight = [x / sum(topicweight) for x in topicweight]

In [85]:
import numpy as np
import scipy as sp
from scipy import stats

def jsd(p, q, base=np.e):
    '''
        Implementation of pairwise `jsd` based on  
        https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence
    '''
    ## convert to np.array
    p, q = np.asarray(p), np.asarray(q)
    ## normalize p, q to probabilities
    p, q = p/p.sum(), q/q.sum()
    m = 1./2*(p + q)
    return stats.entropy(p,m, base=base)/2. +  stats.entropy(q, m, base=base)/2.

In [ ]:
vector_df.set_index('id',inplace=True)
vector_df.head()

In [ ]:
#jsd between archipelago and The Oystermen Seafood Bar & Kitchen(recommended ..)
jsd(np.multiply(vector_df.loc['service_1549650716436w0cpoz1x02p','topicDistribution'],topicperweight).tolist(),
    np.multiply(vector_df.loc['service_1549649911396cbpx9oudge','topicDistribution'],topicperweight).tolist())

In [ ]:

#jsd between archipelago and Laughing Gravy Bar & Restaurant(recommended...)
jsd(np.multiply(vector_df.loc['service_1549650716436w0cpoz1x02p','topicDistribution'],topicperweight).tolist(),
    np.multiply(vector_df.loc['service_1549650037196luup3iff2v','topicDistribution'],topicperweight).tolist())

In [ ]:
#jsd between archipelago and top korean restaurant
jsd(np.multiply(vector_df.loc['service_1549650716436w0cpoz1x02p','topicDistribution'],topicperweight).tolist(),
    np.multiply(vector_df.loc['service_154965134006635lky542nxh','topicDistribution'],topicperweight).tolist())

In [ ]:
#rc and Casa Malevo (100th restaurant of 101 topic)
jsd(np.multiply(vector_df.loc['service_1549650716436w0cpoz1x02p','topicDistribution'],topicperweight).tolist(),
    np.multiply(vector_df.loc['service_1549650129390kwicop4s9w','topicDistribution'],topicperweight).tolist())

In [ ]:
#rc and Buenos Aires Argentine Steakhouse - Wimbledon (101topic restaurant of 38th topic)
jsd(np.multiply(vector_df.loc['service_1549650716436w0cpoz1x02p','topicDistribution'],topicperweight).tolist(),
    np.multiply(vector_df.loc['service_1549650090192fs7p5wr3xz8','topicDistribution'],topicperweight).tolist())

In [ ]:
#rc and  - Wimbledon (101topic restaurant of 55th topic)
jsd(np.multiply(vector_df.loc['service_15496499681925x64n52mbqd','topicDistribution'],topicperweight).tolist(),
    np.multiply(vector_df.loc['service_1549650011117d4ywhsbbgpt','topicDistribution'],topicperweight).tolist())

In [ ]:
service_15496561833503hkcmbqskik

In [91]:
#jsd between the clink restaurant and babur restaurant  68, cuisine - indian(got in output )
#Chamas - Crowne Plaza Dubai and Couqley French Bistro & Bar
jsd(vector_df.loc['service_4ede3c3a674b4080b7bf5401dac588db','topicDistribution'],
    vector_df.loc['service_a97de12289b8450991972f361eeaa90f','topicDistribution'])

0.04976580958989532

In [ ]:
#jsd between the clink restaurant and The Ferm  132, cuisine - english
jsd(vector_df.loc['service_15496498029733if9404ywys','topicDistribution'],
    vector_df.loc['service_15496555117093enqzxxr8tq','topicDistribution'])

In [ ]:
#jsd between the clink restaurant and The Ferm  132, cuisine - english
jsd(vector_df.loc['service_15496555117093enqzxxr8tq','topicDistribution'],
    vector_df.loc['service_15496498029733if9404ywys','topicDistribution'])

In [ ]:
#jsd between the clink restaurant and Hawksmoor Air Street 164, cuisine - english
jsd(vector_df.loc['service_15496498029733if9404ywys','topicDistribution'],
    vector_df.loc['service_1549650021838y4ageh8gb1p','topicDistribution'])

In [ ]:
#jsd between the clink restaurant and the Koffipot 84, cuisine - british
jsd(vector_df.loc['service_15496498029733if9404ywys','topicDistribution'],
    vector_df.loc['service_1549656836515jag92j0z6m','topicDistribution'])

In [ ]:
#jsd between the clink restaurant and Simpson's in The Strand 83, cuisine - british
jsd(vector_df.loc['service_15496498029733if9404ywys','topicDistribution'],
    vector_df.loc['service_15496499310342wjtjwyus8l','topicDistribution'])

In [ ]:
#jsd between the clink restaurant and Golden Pie 4, cuisine -cuisine - English Bar Food
jsd(vector_df.loc['service_15496498029733if9404ywys','topicDistribution'],
    vector_df.loc['service_15496528711969oouoj9fdvv','topicDistribution'])

In [ ]:
#jsd between the clink restaurant and Golden Pie 4, cuisine -cuisine - English Bar Food
jsd(vector_df.loc['service_15496498029733if9404ywys','topicDistribution'],
    vector_df.loc['service_1549650506785snnsx7p25cq','topicDistribution'])

In [ ]:
dff = pd.DataFrame({"id":"service_1549650100958tlmwok7i1pc","name":"Benares Restaurant & Bar","topicDistributions":[2.2935183272516498E-8,1.3567405263968248E-5,1.0109709308037087E-7,5.46877398743438E-7,2.8127366998640047E-4,3.931680129315668E-5,2.7667631452654124E-8,5.755193569728638E-4,1.2474550350435004E-7,3.4641135166839364E-8,6.842257776092415E-8,6.159571259122507E-8,4.0780210400620167E-8,3.3518031253433816E-8,0.056775161321705105,7.729389926455249E-9,5.3168814699131795E-8,3.085094526731196E-8,8.518916465016174E-5,8.101418513156367E-9,3.724889828520553E-8,8.500558206272778E-7,6.445145478253052E-8,4.050859295781449E-8,0.05661760306024717,1.4929561184240744E-7,1.4422920322554074E-7,3.445688181864991E-8,2.409215766365994E-8,0.05799678019971509,3.141972102840756E-4,7.701178805379522E-8,8.445428657183507E-8,0.03698769800715656,0.030245158904604736,8.645631464968752E-5,3.2790525535030617E-7,1.9729392440322565E-5,6.051220702820746E-8,1.807983296218853E-8,3.814282028855522E-8,4.5894998388609536E-5,3.3425057336847624E-8,2.5465222305503682E-8,9.076639724594122E-8,0.01678217215526787,7.978712564956207E-8,1.2272663767025626E-7,6.338378090964927E-8,9.159444018050521E-8,2.4137558423632698E-8,6.288252856359206E-8,7.216595820563655E-5,5.381266361185391E-8,2.110625687273014E-8,9.661246752384739E-8,1.534418640196688E-7,4.127313276748191E-8,0.05231079583520232,6.736541800875151E-4,6.250887181916772E-8,6.930500033429852E-8,9.026240822777557E-8,0.2241729628991798,6.302078989281134E-8,7.196259002010447E-4,1.1159902354970953E-7,2.663752634104072E-8,0.07265397341156551,7.399084134409633E-8,4.1866559673777876E-7,1.3760216842988266E-4,4.7682964617762634E-8,2.6008215945479907E-8,1.0290566531098669E-7,5.658931468094386E-8,1.814997499258722E-7,5.5544823452418156E-8,6.765789016409885E-8,7.104780780855647E-6,1.7577962747309763E-7,2.2756063234601143E-8,1.21330462701312E-7,2.4300610077660115E-8,1.7693652480644145E-8,1.5994405765944878E-7,1.662590562668654E-7,9.528591543504882E-8,0.0034848404251405667,1.011685101989907E-7,5.313759729182334E-7,3.8593306298718674E-8,5.0198594576245144E-8,7.478530198384653E-8,0.17186782177179538,5.3916087403275505E-8,6.493875918454903E-8,8.03521334865816E-4,1.2978385929685838E-7,3.8612656976393294E-8,4.085745827689717E-8,4.351276296973948E-7,4.002368615809431E-8,7.782650553223382E-8,3.1995333527786587E-8,5.88578507901269E-5,1.7637354867992E-8,3.9237702995340484E-4,8.728249293250549E-7,2.0921628867339177E-4,2.5253517792325876E-8,9.949575428098738E-8,1.147528968405478E-8,0.001070971065071802,0.010721378031413306,1.584032662603639E-8,8.444277464149995E-8,3.8089724903820786E-7,0.0015166803055802738,5.1244724889008556E-8,0.05368556694380361,8.907082182823229E-8,4.206041135782648E-8,3.9032214007205545E-7,4.8871319339252684E-8,2.0689812437495663E-7,2.9670476901106535E-8,0.035115261499604485,1.278981748805623E-7,4.0488796124069985E-8,1.1126372217198868E-7,1.4228919584755208E-7,0.003530740973571877,9.106017653667409E-7,1.1910372135821815E-7,6.8397318816616805E-9,8.793053001886216E-8,8.198706791551775E-8,1.0207012773772598E-8,6.654130971342244E-8,1.0069006577306398E-7,2.7394526079229228E-8,1.6315627414480173E-8,6.656285979172471E-7,3.157806759086967E-8,3.045408526453905E-8,1.587216468170256E-8,2.542147483308291E-7,6.445895307970144E-8,9.883273793184137E-8,4.1051649688237795E-8,4.0217151158657914E-8,3.924346390093262E-8,7.77949734073988E-8,8.125476631343452E-8,7.042038459454093E-8,2.4445975741041056E-8,1.1799701242356473E-4,0.04082490970364763,6.800446949407888E-9,4.4118553926611063E-8,7.287499237441517E-8,0.008786791993500388,5.608269076278534E-8,1.5200280915028018E-8,5.002249561587313E-8,0.006550487725289815,6.28737433472108E-8,2.8106099335970795E-7,1.3076787314911622E-4,1.0873309148950175E-7,2.522282317561605E-8,4.2925291232733855E-8,6.115517172480031E-8,0.011173354882377862,0.009941816023379004,2.1577711096220348E-7,8.969984677260092E-8,0.032388087015522524,4.8159673344124685E-8],"ThaKrowdScore":3.85033976,"CuisineList":["[\"Indian\"","\"StreetFood\"","\"VegetarianFriendly\"","\"VeganOptions\"","\"Options\"]"],"priceRange":"NaN","latlong":""})


In [ ]:
ddd = [{"id":"service_1549649854375pj40ofwy2g","name":"Flat Iron Curtain Road","topicDistributions":[4.815845686182414E-7,0.0038537568520341015,2.1227996908143554E-6,0.0011095980391292225,3.713755905492372E-6,1.973596073185505E-6,5.809547802385165E-7,1.4254480477090918E-4,2.6193603416370745E-6,7.273818542256658E-7,1.4367121990356453E-6,1.293364187437058E-6,8.562879049446084E-7,7.037993300651112E-7,3.0224142610033005E-6,1.62298895508486E-7,1.1164192754242216E-6,6.477968364799216E-7,4.334117103607934E-6,1.701106153587828E-7,7.821387079858815E-7,0.0012532284319687838,1.3533280148433452E-6,8.505845814756608E-7,0.025794716875662572,3.1348545146303223E-6,3.028471924214258E-6,7.235129699813174E-7,5.058782926509088E-7,0.08756035561923231,8.708365343406719E-6,1.6170652873242579E-6,1.7733401422344909E-6,1.2351614191536676E-5,0.015053626612038207,3.094126816254402E-5,6.885234317475973E-6,2.477457690780774E-6,9.621211576665887E-4,3.796337031332593E-7,8.009089544341576E-7,2.835803124373095E-6,7.01847097855763E-7,5.347094005330795E-7,0.01606183640955703,0.04930618271017929,1.6753408085157095E-6,2.5769689371186193E-6,1.3309093903489583E-6,1.9232664695998316E-6,5.068316011615332E-7,1.3203842773827104E-6,5.406330463231855E-6,1.1299385788108292E-6,4.4318144269549893E-7,2.0286331677745884E-6,3.2219159975251027E-6,8.666381080620982E-7,1.3473169535344789E-5,6.914020960415501E-6,1.312538370073545E-6,1.455240984669843E-6,1.8952969510780534E-6,0.02515150175807293,1.323287438076806E-6,1.1360798732790174E-5,2.3433153760234534E-6,5.593250108177399E-7,5.357375268173698E-6,1.5536325560169658E-6,0.023343732782455603,6.7682439421465776E-6,1.001229406934863E-6,5.46110981883587E-7,2.160775427356202E-6,1.3845260518435785E-4,3.8110651976830403E-6,4.1295940034957783E-4,1.4206555692686567E-6,1.1919218192667736E-5,3.6909561638405254E-6,4.778234717407764E-7,2.54765256706195E-6,5.102553000059798E-7,3.715248269841772E-7,3.3584466753870957E-6,3.4910467005990955E-6,2.0007787134263114E-6,0.00577659896559661,2.1242992812843256E-6,1.1157637837521737E-5,8.103680944940101E-7,1.0540516823866867E-6,1.5703143492222876E-6,1.211850304909997E-7,1.1321102336602727E-6,1.363560253995019E-6,0.05009002232713185,2.725153889133506E-6,8.107744129284291E-7,8.579099275258762E-7,0.13095933943373456,8.404026862979088E-7,1.6341724262007E-6,6.718262815558136E-7,4.962976999361062E-7,3.703427103565849E-7,3.1271901893950953E-6,9.791778023553281E-4,5.872832213229462E-7,5.302641067919371E-7,2.08917536587439E-6,2.4095392509407735E-7,0.03676335153147929,1.0008504380814699E-5,3.326093702557436E-7,1.7730984189424408E-6,7.997940770142072E-6,1.3645961140909384E-6,1.0760179431051847E-6,0.028743148451459262,1.870276456784334E-6,8.831690952274718E-7,8.195841176207213E-6,1.0261820436377936E-6,4.344370951438407E-6,6.230097945735353E-7,0.04579952177542016,2.6855589791917883E-6,0.004393309801039598,2.3362748406404075E-6,2.987736360641743E-6,1.4448039292527873E-5,1.912048197507315E-5,0.0019242019853321689,1.436181820984881E-7,4.136394235410597E-4,1.7215343895543607E-6,2.143231115762875E-7,1.3972100223865035E-6,2.1142530806619036E-6,5.75220213748012E-7,3.425895619309896E-7,0.0079753097084927,6.63064684413051E-7,6.39463715659987E-7,3.332778940744309E-7,5.33790807148254E-6,1.3534854613379087E-6,2.0752535916859562E-6,0.0013735056225328197,8.444649934411351E-7,8.240198654293342E-7,1.6335103262054282E-6,1.7061577890298596E-6,1.4786614143838753E-6,5.133076349038266E-7,6.899457338309924E-6,0.0060501664279463815,1.427932914957908E-7,9.263852182192028E-7,1.530202368504536E-6,0.018549220913361357,1.1776037765658878E-6,3.1916992510418533E-7,1.0503540209877258E-6,5.847129803715652E-6,0.3310829649727213,1.4316597918530012E-4,5.285811003195549E-7,2.2831375854863164E-6,5.296195924851857E-7,9.013295257117131E-7,1.2841138718588039E-6,0.019246222501543325,0.023996774175217155,4.530808656102855E-6,1.883484491919021E-6,0.035205195534499543,1.0112391619742624E-6],"ThaKrowdScore":4.3049102096,"CuisineList":["[\"Steakhouse\"","\"British\"","\"Options\"]"],"priceRange":"NaN","latlong":"12345.234"},
{"id":"service_1549649920795cwknhz8s6d6","name":"Casa Tua Camden","topicDistributions":[6.233657614112221E-4,1.5656429013441106E-5,3.2097063573555366E-6,1.736267394004513E-5,5.615257054683628E-6,2.984108152251197E-6,8.78412720491925E-7,7.984109841214519E-6,3.960513833282226E-6,1.0998127481531426E-6,2.1723313315378873E-6,1.955586894399895E-6,1.2947207144038166E-6,1.0641556025222305E-6,4.5699376677329245E-6,2.4539847021813345E-7,1.6880434179961269E-6,9.794789557024882E-7,6.553252895772316E-6,2.5720991289634457E-7,1.1826059680614838E-6,6.496258079380495E-4,2.046253139957664E-6,1.2860972000536173E-6,0.02571908021600027,4.739949091067395E-6,4.579096949318966E-6,1.0939629346221288E-6,6.23402493902191E-4,0.08009106990828373,1.3167184697574742E-5,2.445028023813451E-6,2.6813180503621948E-6,1.8675833977990345E-5,0.00408219110795903,0.051103066718227584,1.0410582051765433E-5,3.74595480420853E-6,1.921186946905035E-6,5.740121009466498E-7,1.210986874472418E-6,4.2877787084174866E-6,1.0612037968664763E-6,8.084889825711232E-7,2.8817196755889837E-6,0.029929358779477618,2.533141523949987E-6,3.8964173661643525E-6,2.0123558288386883E-6,2.908009003899546E-6,7.663373135940057E-7,1.996441693300685E-6,8.174456277192729E-6,1.7084848163130668E-6,6.700972777775581E-7,3.067325100678955E-6,4.87158741584074E-6,1.3103703835129087E-6,2.0371643211769713E-5,1.0454107906442037E-5,1.9845785586495815E-6,2.2003471454187284E-6,0.007266971028723965,0.039067193645108,2.000831317708765E-6,0.0039605491595346,3.5431295247768253E-6,8.457081705909552E-7,8.100435256046136E-6,2.349116741264067E-6,1.3292109461416457E-5,1.0233690773201077E-5,1.5138745339544078E-6,8.257283520241081E-7,4.183588582693717E-4,1.7966400175860743E-6,5.762390227505194E-6,1.7634787264524522E-6,2.148053456021033E-6,0.0016783889724760477,5.580783488457546E-6,7.224765678912036E-7,3.852090555795645E-6,7.715139998325811E-7,4.1565348335728414E-4,5.078023937668E-6,5.278517251168149E-6,3.0252087297424033E-6,0.005206028325964724,4.183037057656073E-4,0.12475193599024241,1.2252892422919543E-6,1.5937426411815493E-6,0.1205865224870329,1.8323366282104037E-7,1.7117683924349222E-6,2.0617244457041382E-6,0.016793871794380705,4.120475332916809E-6,1.2259036021242592E-6,1.2971732379336612E-6,1.3814758434156655E-5,1.270702015183668E-6,2.470894285545765E-6,1.0158118527523405E-6,7.504098900705749E-7,5.599639744510237E-7,4.7283605113541054E-6,4.4280283268109753E-4,8.879814224607158E-7,8.017676288593259E-7,3.1588658517776447E-6,3.6432610601499517E-7,0.1902840299498015,0.004581142067315061,5.029105736379365E-7,2.6809525609613008E-6,1.2093011623640306E-5,2.063290682381857E-6,1.6269559711912428E-6,0.005686742717530781,2.8278872751532005E-6,1.3353654948404915E-6,0.002295396766156982,1.5516033111936088E-6,6.568756874179341E-6,9.420005603874569E-7,0.040400718378026085,4.0606072093680925E-6,1.2854686753236852E-6,0.020343027352320613,4.517504139627627E-6,2.1845661542733298E-5,0.04423617992643412,3.7813943010632487E-6,2.1715293916239583E-7,2.7916844353063014E-6,2.602986941475886E-6,3.2405989916586526E-7,2.1126034221789537E-6,3.1967837490383853E-6,8.697419662041338E-7,0.0010382473307896313,2.1132876102324453E-5,1.002564180740986E-6,9.668791465975154E-7,5.039213921151434E-7,8.071000549959698E-6,2.046491201521706E-6,3.1378159113091014E-6,1.3033385725494483E-6,1.2768442871651293E-6,1.2459309336158595E-6,2.4698931799901046E-6,2.5797372808171003E-6,2.235759201709489E-6,7.761291779715222E-7,1.0432087481876938E-5,0.0415251014291039,2.1590569166040252E-7,1.4007089491979793E-6,2.313689931029405E-6,0.01372604758864029,1.7805553413470745E-6,4.825899222222868E-7,1.5881517192726434E-6,8.840951779142894E-6,1.9961627736600652E-6,0.002291927861299987,7.99222896731631E-7,3.452139763616383E-6,8.007931131399491E-7,1.3628243499693147E-6,1.9416002724649367E-6,0.0643833868735177,5.85609184342535E-4,6.850653601648096E-6,2.847858031001444E-6,0.054268535989718474,1.5290094402409292E-6],"ThaKrowdScore":4.119895166,"CuisineList":["[\"Italian\"","\"Cafe\"","\"\"","\"VegetarianFriendly\"","\"VeganOptions\"","\"Options\"]"],"priceRange":"NaN","latlong":"12345.234"},
{"id":"service_1549649990473hhezfdfjb4t","name":"Palm Court Brasserie","topicDistributions":[1.835419889473668E-8,9.47284020516047E-4,8.090435258480664E-8,5.669079282939941E-6,1.4153903380144153E-7,1.05380839906668E-5,2.2141406266276033E-8,0.3521227284505677,9.982932141146529E-8,2.772204944852243E-8,5.475611797789635E-8,4.92928243271167E-8,3.2634929652343954E-8,2.6823269946254222E-8,1.1519055240475975E-7,6.185551619948657E-9,1.0467121009398388E-4,2.4688897350361104E-8,1.651823013784311E-7,6.483272662493039E-9,2.9808948095729085E-8,3.668805825154331E-4,5.157817167091232E-8,4.711531501929238E-5,0.0378960824156025,1.1947588651495827E-7,1.154214226665089E-7,2.757459814806813E-8,1.928008371714445E-8,0.018068824437499915,1.0794760185775097E-5,6.16297527856455E-8,6.758571557219871E-8,0.12333673634655241,0.03639334740944172,1.1642102785478035E-5,2.624107339286407E-7,9.442111349280263E-8,4.842573395461422E-8,1.4468637386877708E-8,3.0524321592025614E-8,1.0807841023726686E-7,2.6748866278458353E-8,2.0378897763332153E-8,7.263706978993299E-8,0.0048349218045424195,6.38507552353528E-8,9.821369599350106E-8,5.072375082828235E-8,7.329972264662854E-8,1.931641630575344E-8,5.032261667789552E-8,0.0032384631049158816,4.306433130495557E-8,1.689057514660038E-8,7.731546870911133E-8,1.227939823952028E-7,3.3029397621208466E-8,0.007204196801431419,5.914154425694948E-4,5.002359267952432E-8,5.546228889567256E-8,6.330756328664366E-4,0.03615565809658532,5.043326222651784E-8,4.329838892948387E-7,5.240363917413E-5,2.131705034653531E-8,2.0418081814359606E-7,5.921219823195432E-8,4.659325844595517E-4,2.5795198512834895E-7,3.815895456714526E-8,2.081343596380266E-8,8.23516876264236E-8,4.5286384879928745E-8,1.4524769520705518E-7,4.4450516270361485E-8,5.4144166109986565E-8,5.685699477779627E-6,1.406700877838661E-7,1.8210855597111026E-8,9.709638758713818E-8,1.9446900656045123E-8,1.415959109396997E-8,1.2799745314573406E-7,1.3305111847220226E-7,7.625387697937732E-8,5.669558249642095E-6,8.096150512568946E-8,6.595858053748679E-4,3.08848292771656E-8,4.017212237378211E-8,5.984797639089706E-8,4.618615914289675E-9,4.314709763019678E-8,5.196814378539925E-8,0.016347791123884948,1.0386133559767676E-7,3.090031492568072E-8,3.2696748338008574E-8,0.004373826247217461,3.20295106220697E-8,6.228174176103769E-8,2.8775442249215463E-4,1.891494721636021E-8,1.4114538147689202E-8,6.749740473085324E-4,2.6855655522779602E-5,2.238259644122394E-8,2.0209478286905507E-8,7.962285897420343E-8,9.183259917013595E-9,0.00980280998296706,5.287561832218308E-4,1.267644134329888E-8,6.757650299009189E-8,1.310906445773709E-4,5.200762258822339E-8,4.100930268327113E-8,0.10646175479947559,7.128016201693304E-8,3.365942824538349E-8,3.1236071265611126E-7,3.910995193466421E-8,1.6557309704502992E-7,2.3744211148169622E-8,0.01071742360300104,1.0235229045867945E-7,3.240172133091854E-8,8.904033869049322E-8,0.03782337482463868,0.020021244929699106,1.119158780863918E-5,9.531440690651059E-8,5.4735904156965454E-9,7.036762766224584E-8,6.561128958236514E-8,8.16830370810395E-9,5.3250607099352996E-8,8.057862333010075E-8,2.192284989881641E-8,1.3056807399338911E-8,5.76411157907321E-6,2.5270787086702286E-8,2.4371304622293035E-8,1.270192018945307E-8,2.0343888241044402E-7,5.158417228734567E-8,7.909227093406903E-8,5.2642852032970955E-6,3.21843336752506E-8,3.1405126926509416E-8,6.225650774027932E-8,6.502525505646377E-8,5.635489063830087E-8,0.007465274526417191,2.629528031108392E-7,4.0148938492013006E-7,5.4421521031448254E-9,3.871613522287497E-4,5.831922459914463E-8,7.062873356186658E-7,4.488095205437749E-8,1.2164235875307351E-8,4.0031196735411587E-8,2.228463916289088E-7,5.0315586186493764E-8,2.984166068461641E-4,2.0145335377127762E-8,8.701516634616984E-8,2.0184914493654075E-8,3.435156037511132E-8,4.8940275381353144E-8,1.1133158153606303E-6,0.004869530826199783,1.7267862935622414E-7,7.178354796338534E-8,0.1565792882507405,3.854044734505519E-8],"ThaKrowdScore":4.1648283295,"CuisineList":["[\"French\"","\"\"","\"British\"","\"Soups\"","\"VegetarianFriendly\"","\"VeganOptions\"","\"Options\"]"],"priceRange":"NaN","latlong":"12345.234"},
{"id":"service_1549650090192ei53mcx4b0c","name":"ROKA Aldwych","topicDistributions":[1.9957923612306703E-7,4.291204584639743E-6,8.797348759545872E-7,6.304979421780792E-4,1.5390621192292543E-6,8.179016155604619E-7,2.407604371434081E-7,0.013540906609924755,1.0855205298934527E-6,3.0144303679135735E-7,5.954051310965635E-7,5.35998562616406E-7,3.5486453907898984E-7,2.916699201582365E-7,0.02133326664990299,6.726023153552972E-8,4.62668699749933E-7,2.6846125522366875E-7,1.7961534425570192E-6,7.049758003488647E-8,3.2413548118865563E-7,7.39708461567241E-6,5.608488914031015E-7,3.5250095640739134E-7,0.05870115100065312,1.2991526518010806E-6,1.2550653667932798E-6,2.998396860769624E-7,2.0964708962372158E-7,0.10068117685543704,3.60893811050546E-6,6.701474171634803E-7,7.349111537958963E-7,0.04215717863220553,0.045091262261925245,1.2822741979207188E-5,1.7350950527923753E-4,1.0267129506909117E-6,5.265700261171456E-7,1.5732855538809556E-7,3.319142841073976E-7,1.1752191790149306E-6,2.9086087219767895E-7,2.2159533477662367E-7,7.898383898983414E-7,0.038864426622415356,0.002844962844070066,0.029809002314630764,1.1432229996020142E-4,7.970439209984751E-7,2.100421616354472E-7,5.471962821089604E-7,1.7289661499835424E-4,4.6827139638648994E-7,1.8366413618871964E-7,8.407101979202295E-7,1.335232844947086E-6,3.59153890869962E-7,0.08573183755405357,2.8653223374199546E-6,5.439447655748064E-7,6.030838673438053E-7,0.0062581762539271725,0.14148723005356154,5.483994157461792E-7,4.708164838719227E-6,5.7856492345337964E-5,2.3179658501894622E-7,2.220214129191303E-6,6.438594983129393E-7,3.6431782120555885E-6,2.8049091351575014E-6,4.149314850210972E-7,2.2632040082900618E-7,8.954728563304506E-7,4.924334836375358E-7,1.5793892299147402E-6,4.83344445235292E-7,5.887509106077279E-7,4.939586143513913E-6,1.5296134048824996E-6,1.9802055486394147E-7,1.0558032511300354E-6,2.1146101772971655E-7,1.5396805878353498E-7,1.3918141604016961E-6,0.1401670007131773,8.291666994700936E-7,0.0031903448417450013,8.803563392239623E-7,4.623970495856271E-6,3.358343597712484E-7,4.3682219762263443E-7,6.507728002790584E-7,5.0221741706422684E-8,4.691713779145909E-7,5.650893563324054E-7,0.02109971904515541,1.129363702561273E-6,3.3600274706612527E-7,3.5553674090778305E-7,0.005578552779015165,3.482813551280327E-7,6.772369917298075E-7,2.784195857126787E-7,2.0567668130870296E-7,1.5347816365624682E-7,1.2959763868716745E-6,7.595218690604823E-6,2.43383082302293E-7,2.1975310729050354E-7,8.658002162417015E-7,9.985660555758337E-8,3.8845955500301416E-4,6.867721953588222E-4,1.3784063769625794E-7,7.348109783477524E-7,3.3145225438646414E-6,5.655186395365312E-7,4.459254991409047E-7,0.08597654576815211,7.750836943445932E-7,3.6600469549663776E-7,3.3965368243143854E-6,4.252724064229518E-7,0.0026185274651783106,2.5818895994722153E-7,0.03502200363480279,1.1129547011200454E-6,3.523286867154757E-7,9.682036727347317E-7,1.23818365340248E-6,5.987585220498589E-6,7.923948223338956E-6,9.112023611313977E-4,5.951853307683849E-8,7.651610106857191E-7,7.13441710586064E-7,8.88202107776053E-8,5.790345603711469E-7,5.128445312519137E-4,2.3838390667658766E-7,1.4197664860836495E-7,5.792220865886117E-6,2.747885871738597E-7,2.6500782669597406E-7,1.3811768866873098E-7,2.2121464947647546E-6,5.609141406929726E-7,8.600307268538303E-7,3.5722657146533807E-7,3.499648645454308E-7,3.4149195387318674E-7,6.76962602931029E-7,7.070693091703487E-7,0.006371774332838035,2.1272597243757174E-7,0.0017663057853692507,4.365701015523901E-6,5.917668027029242E-8,3.8391440726260457E-7,6.341495133369509E-7,0.049896150298905706,4.880249025773218E-7,1.3227103606852783E-7,4.3528980542085823E-7,2.4231791792794306E-6,5.471198342012321E-7,2.4457593249729535E-6,2.190556323960899E-7,5.783155334552442E-5,2.1948600638778792E-7,3.7353077726896313E-7,5.321650284101455E-7,1.2105933975528705E-5,0.014381849881032692,1.8776667475024583E-6,0.005746203020305159,0.03779903858869436,4.190797476414595E-7],"ThaKrowdScore":3.9096249979,"CuisineList":["[\"Japanese\"","\"Sushi\"","\"\"","\"VegetarianFriendly\"","\"VeganOptions\"]"],"priceRange":"NaN","latlong":""},
{"id":"service_1549650327042b22d3unavlq","name":"Briciole","topicDistributions":[3.542735113899911E-7,7.617326059689438E-6,1.5616191826913596E-6,8.447465801802771E-6,2.7319923245444774E-6,1.4518588353423473E-6,4.2737434578609537E-7,3.884510823297837E-6,1.926909719075877E-6,5.350921528845026E-7,1.0569048694180158E-6,9.514521478627654E-7,6.299207711657037E-7,5.177438735038528E-7,2.223412839396088E-6,1.1939377495313435E-7,8.212841578849277E-7,4.7654612477625983E-7,3.188355659891966E-6,1.2514039891432144E-7,5.75373557477592E-7,0.0029414733993262275,9.95564017461413E-7,6.257251707175117E-7,0.10490737491069689,2.3061285368452804E-6,2.227869106802499E-6,5.322460417428202E-7,3.7214497878872145E-7,0.08804171348043772,6.406233442087798E-6,1.1895800547159187E-6,1.304542296446088E-6,9.086357861351948E-6,0.06156273916291676,0.01819868262267578,0.0019236344962176284,1.8225202646158837E-6,9.34715533384484E-7,2.7927424135888497E-7,5.891817264934015E-7,2.0861339751082507E-6,5.163077308096655E-7,3.9335433326614263E-7,1.4020437459103057E-6,2.227558273643722E-4,1.2324499364893594E-6,1.8957248499788532E-6,4.0488842633051733E-4,0.031910253830298475,3.7284627192704996E-7,9.713292427011344E-7,3.977120119191624E-6,0.005150675497857873,3.2602258485379016E-7,1.4923463966708213E-6,2.370174561701205E-6,6.375348083275536E-7,0.010511554634196665,5.0862395580870735E-6,9.65557469032319E-7,1.070535409880003E-6,1.3942587652085598E-6,0.01658391471009393,9.734649276882543E-7,2.103121502517847E-4,1.723839634106044E-6,4.114625934946118E-7,3.941106654508157E-6,2.0309759350591687E-4,0.001319172414850736,4.978999958840209E-6,7.365457301081316E-7,4.0174180770709584E-7,1.589555703936014E-6,8.74119686813012E-7,2.8035770614353943E-6,8.579857161030738E-7,1.0450929489801135E-6,0.0038459071361022523,2.715219892327723E-6,3.515066930899768E-7,1.8741585165911702E-6,3.7536488629013894E-7,2.7330901694356705E-7,2.4706121708160574E-6,2.5681582293972985E-6,1.471855508895638E-6,1.0943404943196858E-4,1.5627223434154827E-6,0.21559482591969795,5.961402608445927E-7,7.754039789461698E-7,1.1551881325459819E-6,8.914871671061141E-8,8.328270752323607E-7,1.0030912754548459E-6,0.033617015687126224,2.0047358248032714E-6,5.964391654771002E-7,6.311139980107172E-7,6.721297643190173E-6,6.182349478318892E-7,1.2021647730697116E-6,4.942231776523821E-7,3.650970988452072E-7,2.7243940309831615E-7,2.300490342454398E-6,0.0028408477689340884,4.320298085868207E-7,3.9008419147702134E-7,1.5368837396542667E-6,1.7725566533662867E-7,0.05541927287129255,0.030603496263714957,2.446811986933184E-7,1.3043644748026226E-6,5.8836157658812414E-6,1.0038532969509728E-6,7.91563268142621E-7,0.0493173253530283,1.3758526555701592E-6,6.496956856717409E-7,0.0017226439555972225,7.549019208983594E-7,3.1958988141225575E-6,4.5831175236199857E-7,0.046314188324779564,1.9756081726891885E-6,6.254193744340577E-7,1.7186603253101379E-6,4.0610725672247813E-4,0.0032419034098436993,1.4065816743700407E-5,1.8397626512866867E-6,1.0565147014046437E-7,1.3582388794553248E-6,1.0224377043870455E-4,1.5766493832643998E-7,1.0278454357463924E-6,1.5553319430526853E-6,4.2315576168004836E-7,2.5202304014658426E-7,1.0281783139196408E-5,4.877777846987478E-7,0.0034336999281645115,2.45172992442802E-7,0.023632624016994935,9.956798416096931E-7,1.526642307208318E-6,6.341136196993681E-7,6.212233488516824E-7,6.061830677389785E-7,1.2016777049502135E-6,1.255120180946132E-6,1.0877644458868232E-6,3.77610309986544E-7,5.075525955787267E-6,7.749564826948162E-6,1.0504464652240562E-7,6.814872517613075E-7,1.1256801017997757E-6,0.04131336427289254,8.662940055307852E-7,2.3479458742037696E-7,7.726838264057765E-7,4.301390331212017E-6,9.71193539863327E-7,4.341472394145704E-6,3.8884610236138855E-7,1.6795703643857696E-6,3.896100601167723E-7,6.630552488622012E-7,9.446472334304439E-7,2.1489268229894056E-5,0.020884293646415232,3.333050094688864E-6,1.385569032071079E-6,0.12324055118547196,7.439093195938504E-7],"ThaKrowdScore":3.5703424517,"CuisineList":["[\"Italian\"","\"\"","\"\"","\"VegetarianFriendly\"","\"VeganOptions\"","\"Options\"]"],"priceRange":"NaN","latlong":""},
{"id":"service_1549650327042xbkojo6pi8","name":"McGlynn's Free House","topicDistributions":[1.2674049569248534E-6,0.022424378647219464,5.586655025961626E-6,3.9146460303742696E-4,0.026019341458675165,5.1939900260915225E-6,1.5289214318807703E-6,1.3896743940464449E-5,6.893473124540351E-6,1.9142792931838987E-6,3.7810517225597246E-6,3.403797150249308E-6,2.253526391834821E-6,1.8522162414053853E-6,7.954205898388195E-6,4.2712835517357267E-7,2.938124300242614E-6,1.7048322872773064E-6,1.1406265605158798E-5,4.4768676402953156E-7,2.0583850482342025E-6,0.020999126190599935,3.5616063016284085E-6,2.2385167322516043E-6,0.028870614192196952,8.250119314413249E-6,7.970148087738193E-6,1.9040973991021917E-6,1.331339701240913E-6,0.03211306007557975,2.2918145892036414E-5,4.2556940034565865E-6,4.666968655225898E-6,3.2506226470862026E-5,0.002409086560851958,0.004055113319158869,0.01988654000458166,6.520022364662245E-6,3.645879198415157E-4,9.990968741945819E-7,2.10778344042003E-6,7.463094066750606E-6,1.8470784754958681E-6,1.4072156562125196E-6,5.015777743078574E-6,0.005493075576805274,4.409059973295155E-6,6.781910006204043E-6,3.502606330152689E-6,5.061535638594088E-6,1.333848560557794E-6,3.4749069784398833E-6,7.367160465198788E-4,2.973703580067795E-6,1.1663379474574542E-6,5.338833301898896E-6,8.479242426257687E-6,2.2807654264532338E-6,3.545786756327995E-5,1.819589955437169E-5,3.4542585971104304E-6,3.829814652871131E-6,4.987927162062101E-6,0.004780711947188928,3.482547340059034E-6,2.9898658650705087E-5,6.166994785010761E-6,1.471997521175556E-6,1.4099214163924876E-5,4.088755601919318E-6,2.313558371409263E-5,1.7812252470143334E-5,2.6349746151731468E-6,1.437221644074374E-6,0.005063102562630247,3.1271421328275775E-6,1.0029729433743996E-5,3.069423241080838E-6,3.7387948615963895E-6,3.9261227052461516E-4,9.713633788694102E-6,1.2575067310750287E-6,6.704756967776186E-6,1.3428588427994353E-6,9.777564274773547E-7,0.07045141807323847,9.187524230459437E-6,5.265527644255557E-6,3.0253679401781517E-5,5.5906015568906615E-6,3.9060798969813834E-4,2.1326774295162773E-6,0.002170237974224281,4.132657729970227E-6,3.189273875408076E-7,2.9794188091267565E-6,3.5885348858612477E-6,0.07185736853346726,0.01048324782241485,2.1337467536421523E-6,2.2577951321427614E-6,2.4045280485475287E-5,2.2117206402252386E-6,4.300715530356863E-6,1.76807151831978E-6,1.306126080439362E-6,9.746454048842254E-7,8.229948809734326E-6,0.3724907940115935,1.5455762379577712E-6,1.3955168026992606E-6,5.498164574067712E-6,6.341278748423047E-7,0.17730896247477249,2.6339772680100336E-5,8.753410969776158E-7,0.0014496423226089145,0.023501908332966785,3.5912609994159566E-6,2.8317985327986275E-6,0.008814329152932348,4.922079748006549E-6,2.324270672346742E-6,2.1569315982647514E-5,2.7006434457512894E-6,1.1433251057797329E-5,1.639599259006084E-6,0.050451058384202906,7.067690669797756E-6,2.237422753410403E-6,6.14846593249898E-6,7.862942710823678E-6,3.992674679831524E-4,0.005830224081154152,6.5817065878209285E-6,3.779655905886214E-7,4.859066888054027E-6,4.530629428384789E-6,5.640425206619801E-7,3.6770922982823014E-6,5.564162577536211E-6,1.5138295488149694E-6,9.016063580024015E-7,3.678283161932175E-5,1.745013280218055E-6,1.6829016870835715E-6,8.771004772751566E-7,1.4047981580508433E-5,3.5620206597301073E-6,0.0014504375162024187,2.268526206510075E-6,2.2224115729198813E-6,2.1686053293737753E-6,4.298973055886813E-6,4.490162226993096E-6,3.891451114350869E-6,1.3508917920088036E-6,1.8157571900102476E-5,2.7723881577114378E-5,3.7579469370590157E-7,2.4380042345660345E-6,4.027093460459646E-6,0.0018549909568977814,3.099145946464409E-6,8.399719831950122E-7,2.764257784557616E-6,1.5388120342549452E-5,3.4744215048050326E-6,1.553151295761292E-5,1.391087568698358E-6,6.008622538486365E-6,1.3938206091739376E-6,2.3720641879937563E-6,3.3794527327140107E-6,7.68773396814136E-5,0.021971434036501786,1.1923906461743058E-5,4.956839850991098E-6,0.004146111513796909,2.661317679260391E-6],"ThaKrowdScore":3.9530865217,"CuisineList":["[\"British\"]"],"priceRange":"NaN","latlong":""},
{"id":"service_1549650497413ptjgtho9j9","name":"Tredwells","topicDistributions":[4.824713945994407E-8,4.960990508507353E-4,2.126708773541191E-7,0.006079407680179516,3.7205946944391874E-7,1.977230401168886E-7,5.8202459398793896E-8,0.021480705098801164,2.6241838284272594E-7,7.287213097826224E-8,1.4393578687448443E-7,1.2957458853553754E-7,8.578647377814048E-8,7.050953589915431E-8,0.016956165224032663,1.6259776487979882E-8,1.1184751336968374E-7,6.489897382100942E-8,4.342098272240617E-7,1.704238698175185E-8,7.835789969244031E-8,9.644081297846026E-4,1.3558201346554304E-7,8.521509117844256E-8,0.08014388439910121,3.140627271093078E-7,9.354199052595541E-4,7.248453010843573E-8,5.068098549194732E-8,0.047648252915235036,1.4624153399021446E-5,1.6200430727508327E-7,2.614602121475961E-4,0.05176268607470651,0.09180188388881386,4.981615012462743E-4,6.897913304879789E-7,2.4820198673758635E-7,1.2729529373403585E-7,3.80332789136458E-8,8.023838083137272E-8,2.8410251851535643E-7,1.3822027194085045E-5,5.356940545682883E-8,1.9093891573342166E-7,0.029101458045955566,1.678425906862233E-7,1.4009884677024906E-5,1.3333602268227477E-7,1.9268081168725096E-7,5.077649189239618E-8,1.322815732123276E-7,5.416286086044894E-7,1.1320193325437074E-7,4.4399755027738316E-8,2.0323688451960372E-7,3.227849075538888E-7,8.682340005399906E-8,0.026995962889901665,3.719889328002195E-4,1.314955376771334E-7,1.457920774676018E-7,0.0018291677397497524,0.1452350617762917,1.3257242389154382E-7,0.0123226732357874,2.3476305328889737E-7,2.888420135584045E-4,5.367240741215085E-7,1.5564935316496904E-7,8.807175069358838E-7,6.780707494689826E-7,1.0030731459353842E-7,5.47116631646706E-8,2.164754441455459E-7,1.1904297972739013E-7,3.8180831792663105E-7,8.262712520208326E-5,1.4232716710942983E-7,1.194116713592223E-6,3.6977529676312973E-7,4.787033717558088E-8,2.752894992186876E-4,5.1119492242450595E-8,1.5130606654803014E-4,1.9286044848980144E-4,2.785317401942058E-5,2.004463097564258E-7,0.0015138401432444784,2.1282111254696394E-7,1.1178184349532116E-6,8.118603671442619E-8,1.0559926922910309E-7,1.573206043961461E-7,0.08455929685914702,1.1341949864414346E-7,1.366071216220497E-7,0.008029851385250133,2.7301721921048495E-7,5.914048961023178E-4,8.594897472768323E-8,0.01529282047142018,8.41950266900454E-8,1.6371817139971768E-7,1.3819019584196545E-5,4.9721161978195106E-8,3.7102468722813276E-8,3.132948832193938E-7,1.5587813942124866E-5,5.8836468873891E-8,5.31240574929506E-8,2.0930225302448245E-7,2.4139763574215418E-8,2.3142500924460576E-4,0.04835202644850303,3.3322186212187516E-8,1.776363535492278E-7,8.012668778747925E-7,1.3671089838269678E-7,1.0779994033312054E-7,0.06506485509967519,1.873720524269014E-7,8.847954291062489E-8,4.408759170703245E-4,1.0280717322969409E-7,0.002035688796754245,6.241570515837728E-8,0.02899767704794402,2.690504369123247E-7,8.517344596518276E-8,2.340577032535482E-7,2.9932382026903645E-7,0.06429070686573299,5.794875253046434E-4,2.505501606980514E-7,1.4388265140161174E-8,1.8497330037430113E-7,1.7247045522291437E-7,2.147177822449567E-8,1.3997829498218647E-7,2.1181464250191395E-7,5.762794674361574E-8,3.4322043207136764E-8,0.0014453301265788124,6.642857015046916E-8,6.406412721558453E-8,3.338916170105138E-8,5.347737696166375E-7,1.3559778710837897E-7,1.395962075285114E-5,8.635748160524117E-8,8.46020054681045E-8,8.255372774755379E-8,1.6365183947612642E-7,1.7092996360779877E-7,1.4813843325276915E-7,5.142528781209978E-8,2.3447034134709907E-4,1.0553832644520106E-6,1.4305624178342724E-8,9.280911335114494E-8,1.5330201980308768E-7,0.037764061156143815,1.1797723045725337E-7,0.0026266092047798114,1.513740744721348E-4,5.857897147528719E-7,0.0035343225660050864,0.0036172918306940495,5.295544692426482E-8,2.287341926450296E-7,5.3059487376572106E-8,9.029893015706378E-8,1.2864785355515177E-7,2.92654033634064E-6,0.011548985668884187,4.539152027327567E-7,1.886952881671235E-7,0.0830806636011087,1.0131013336892284E-7],"ThaKrowdScore":3.5114870861,"CuisineList":["[\"British\"","\"Healthy\"","\"VegetarianFriendly\"","\"VeganOptions\"","\"Options\"]"],"priceRange":"NaN","latlong":""}]

In [ ]:
sandf

In [ ]:
latlongdict = {}
latlongdict[x.id] = sandf.apply(lambda x : x.name, axis=1)

In [ ]:
dict(zip(sandf.id, sandf.name))

In [ ]:
sandf = pd.DataFrame(ddd,columns=['id','name','topicDistributions','ThaKrowdScore','CuisineList','priceRange','latlong'])
sandf.head()

In [ ]:

modeljson = []
for row in range(sandf.shape[0]):
    sandf.loc[row,'latlong'] = mydictionary[sandf.loc[row,'id']]
    modeljson.append(sandf.loc[row,:].to_dict())
    

In [ ]:
import json
with open('.json', 'w') as fout:
    json.dump(modeljson, fout)

In [ ]:
sandf.loc[row,'latlong'] = mydictionary[sandf.loc[row,'id']]

In [ ]:
l = [1.8146308234160502e-07, 3.901684493906901e-06, 7.998798138283791e-07, 0.0004181011881636999, 1.3993587784769768e-06, 7.43659265837955e-07, 2.189061942446505e-07, 1.989692383772347e-06, 9.869859466647767e-07, 2.740805289626586e-07, 5.413591735773874e-07, 4.87345042462567e-07, 3.226528687347943e-07, 2.6519453509485694e-07, 0.0038285510989060005, 6.115490353876842e-08, 0.002121013939874, 2.440925609723423e-07, 0.006828909002034001, 6.409839229400645e-08, 0.00387942874073, 6.725638401952185e-06, 5.099396637590203e-07, 3.205038325660567e-07, 0.048407454049393006, 0.00010462480037901241, 1.141140904347729e-06, 2.726227171764199e-07, 1.906170592997388e-07, 0.144143229039137, 0.001503213172533, 6.093169725670447e-07, 6.682019923795831e-07, 0.009366297593206, 0.00168969195066, 0.000942650966047896, 2.594385126330358e-06, 9.335164335308512e-07, 4.787723505915996e-07, 1.430475692545055e-07, 3.017858482542527e-07, 1.068542493666613e-06, 5.198624596035411e-05, 2.0148074149502378e-07, 7.181433878938372e-07, 0.019354602946038, 6.31275433011086e-07, 9.710126879061337e-07, 5.014922321598666e-07, 7.246948604254401e-07, 1.90976269939018e-07, 4.97526325514791e-07, 2.0371279603708833e-06, 4.257655887023352e-07, 1.6699262366082062e-07, 7.643974735756256e-07, 0.0007253190499247631, 3.265528686163316e-07, 0.013814793884926001, 2.605232053949035e-06, 4.945699547818736e-07, 0.003724517007383, 7.141558287095053e-07, 0.101904024628165, 4.986202486228241e-07, 0.001917986916819, 8.829710475394715e-07, 2.107560065409748e-07, 0.0028984387553710003, 5.854152537528508e-07, 3.31248060029515e-06, 2.550302662948102e-06, 3.772674337027657e-07, 2.057769050979564e-07, 8.14189230400526e-07, 4.477344413422858e-07, 1.436025327307773e-06, 4.394704307366166e-07, 5.353089682356846e-07, 5.6212998367464156e-05, 1.3907677403367659e-06, 1.8004588528662378e-07, 9.599661568912095e-07, 1.922663340021279e-07, 1.399921107613884e-07, 1.2654767725308171e-06, 1.31544101736238e-06, 7.539017985221569e-07, 0.020227550357805, 8.0044486579778e-07, 4.204244665598955e-06, 0.001034741090782, 3.971710883138079e-07, 5.917010232963151e-07, 4.566302701445247e-08, 4.265838773445414e-07, 5.137952143242028e-07, 0.0002542868584936251, 1.026849398427023e-06, 3.055031943316408e-07, 0.0015002550880550002, 0.006882440403734, 3.166672518210897e-07, 6.157630141407437e-07, 2.531469565127573e-07, 1.870070518410783e-07, 1.3954668426538237e-07, 1.178338360102029e-06, 6.905787502893566e-06, 2.212907773489073e-07, 1.998057361963782e-07, 7.872100273716234e-07, 9.079244809552918e-08, 0.0161800652393, 0.017175404541831002, 1.253286036873674e-07, 6.681109100307965e-07, 0.0031580426666840003, 5.141855307465488e-07, 4.054480673476976e-07, 0.075172441784374, 7.047280016732779e-07, 5.2054568848344094e-05, 3.0882272795492406e-06, 3.866696917145947e-07, 0.00015680233827241363, 0.002275993382219, 0.13445902103458202, 1.011929870536672e-06, 3.203472000931788e-07, 8.803181443212304e-07, 1.125791573392212e-06, 0.002074315563095, 7.204677685186217e-06, 0.20745703014369402, 5.41159324906625e-08, 6.957060172383359e-07, 6.486813678061338e-07, 8.075784603162163e-08, 0.00015569183569157522, 7.966594153295529e-07, 0.0037759071989060004, 5.185087621623957e-05, 5.6988237955703336e-05, 2.4984555001561223e-07, 2.409526061481066e-07, 0.0065170707468570005, 2.0113461166053322e-06, 5.099989902575609e-07, 0.0016041573623150002, 3.2480049534036525e-07, 3.181979461656846e-07, 3.104941363062141e-07, 6.155135320896367e-07, 6.428874003309769e-07, 5.57165813604792e-07, 1.93416466574879e-07, 2.599744420146899e-06, 3.969418754415604e-06, 0.0011379331198690002, 3.490658299456763e-07, 5.765866609720514e-07, 0.056383730742657005, 4.437260348392869e-07, 1.2026456446958268e-07, 0.0017589365369680002, 0.0005711428803777909, 4.974568169163798e-07, 0.0030538091885410003, 1.991715723091182e-07, 8.602958040392321e-07, 1.9956288050635483e-07, 3.396247400750896e-07, 0.0016555810445890002, 0.00104544279802, 0.013852211202425002, 0.008432358514305, 0.004552226963883001, 0.039540132931037, 3.81039151322675e-07]

In [ ]:
l[134]

In [ ]:
topicMatrix = [ [ 0.000010462,
    4.2169e-7,
    2.6711e-8,
    5.2758e-8,
    4.7494e-8,
    0.043778,
    4.0997e-8,
    0.000065687,
    6.2468e-9,
    6.5546e-7,
    1.1512e-7,
    2.6569e-8,
    0.04472,
    0.00024238,
    0.028521,
    0.000066665,
    0.01294,
    6.1521e-8,
    9.4631e-8,
    1.8612e-8,
    4.8488e-8,
    0.000055741,
    1.6274e-8,
    1.1831e-7,
    0.040335,
    0.00051949,
    8.6052e-8,
    3.2283e-7,
    0.0001061,
    3.6767e-8,
    7.9349e-8,
    4.3635e-8,
    1.282e-7,
    0.0026871,
    7.8008e-8,
    4.0974e-7,
    2.9759e-8,
    3.8707e-8,
    0.13253,
    5.0073e-8,
    0.00061958,
    2.9773e-8,
    3.1504e-8,
    3.6137e-7,
    2.467e-8,
    0.000045383,
    0.00016132,
    1.9472e-8,
    0.0008258,
    0.0082677,
    6.5112e-8,
    2.937e-7,
    3.9514e-8,
    0.041396,
    3.0097e-7,
    2.2878e-8,
    9.8619e-8,
    3.122e-8,
    1.0972e-7,
    0.0029393,
    7.0215e-7,
    9.1839e-8,
    2.1123e-8,
    1.2581e-8,
    6.7278e-7,
    2.4349e-8,
    4.9703e-8,
    7.6208e-8,
    3.0259e-8,
    0.000091052,
    0.031479,
    0.0067753,
    3.8572e-8,
    0.60083,
    2.1672e-7,
    8.3841e-8,
    1.9448e-8,
    4.7155e-8,
    1.6639e-7,
    6.9165e-8,
    3.7135e-8 ],
  [ 0.00054867,
    0.0000090438,
    0.0011897,
    0.0000011315,
    0.0000010186,
    0.0000023804,
    8.7925e-7,
    0.00021962,
    1.3397e-7,
    0.000014057,
    0.0000024689,
    5.6981e-7,
    0.20971,
    0.0000092435,
    0.0000097276,
    0.0011054,
    0.054614,
    0.0000013194,
    0.0000020295,
    3.9917e-7,
    0.0000010399,
    0.0000063208,
    3.4904e-7,
    0.0000025375,
    0.0073619,
    0.0000063773,
    0.001191,
    0.10249,
    0.00011344,
    7.8853e-7,
    0.0000017018,
    9.3581e-7,
    0.0000027493,
    0.012225,
    0.000001673,
    0.002279,
    6.3822e-7,
    8.3014e-7,
    9.544e-8,
    0.0000010739,
    0.12247,
    6.3853e-7,
    6.7567e-7,
    0.31113,
    5.2911e-7,
    3.9087e-7,
    4.6253e-7,
    4.1761e-7,
    0.064629,
    0.0069421,
    0.0000013964,
    0.0000062989,
    8.4743e-7,
    0.040907,
    0.0000064549,
    4.9065e-7,
    0.0000021151,
    6.6957e-7,
    0.0000023531,
    0.00066364,
    0.000015059,
    0.00011007,
    4.5302e-7,
    2.698e-7,
    0.030717,
    5.2221e-7,
    0.000001066,
    0.0000016344,
    6.4898e-7,
    0.000115,
    0.0033595,
    0.012555,
    8.2722e-7,
    0.0002296,
    0.0000046479,
    0.0000017981,
    4.1711e-7,
    0.0000010113,
    0.011138,
    0.0018393,
    7.9642e-7 ],
  [ 0.0000029208,
    0.000003239,
    2.0517e-7,
    4.0524e-7,
    3.6481e-7,
    0.0077057,
    0.000039033,
    0.00023353,
    4.7983e-8,
    0.0000050345,
    8.8424e-7,
    2.0408e-7,
    0.097138,
    0.00062279,
    0.0097603,
    0.00012488,
    0.031446,
    4.7255e-7,
    7.2686e-7,
    1.4296e-7,
    3.7243e-7,
    0.0012799,
    1.2501e-7,
    9.0879e-7,
    0.20451,
    0.000002284,
    6.6097e-7,
    0.0000024796,
    0.0000019091,
    2.824e-7,
    6.0948e-7,
    3.3516e-7,
    9.847e-7,
    0.050645,
    5.9919e-7,
    0.0000031471,
    2.2858e-7,
    2.9732e-7,
    0.0003485,
    3.8462e-7,
    0.024233,
    2.2869e-7,
    0.0019362,
    0.0000027757,
    1.895e-7,
    1.3999e-7,
    1.6565e-7,
    1.4957e-7,
    0.0072336,
    0.002138,
    5.0013e-7,
    0.0063519,
    3.0351e-7,
    0.13897,
    0.0000023117,
    1.7573e-7,
    7.5749e-7,
    2.398e-7,
    8.4274e-7,
    0.0000053716,
    0.0000053932,
    7.0541e-7,
    1.6225e-7,
    9.6633e-8,
    0.000082602,
    1.8703e-7,
    3.8177e-7,
    5.8535e-7,
    2.3243e-7,
    0.28952,
    0.11863,
    0.0027155,
    2.9628e-7,
    0.000082231,
    0.0038735,
    6.44e-7,
    0.0003099,
    3.6219e-7,
    0.000001278,
    5.3126e-7,
    2.8523e-7 ] ]

In [ ]:
topicMatrix= np.array(topicMatrix)
topicMatrix.shape

In [ ]:
topicMatrix[1]

In [ ]:
topicMatrix2 = np.array([ [ 0.0071159,
    0.000016972,
    0.000001075,
    0.0000021234,
    0.0000019115,
    0.000004467,
    0.00000165,
    0.0014265,
    2.5142e-7,
    0.000026381,
    0.0000046332,
    0.0000010693,
    0.25275,
    0.000017347,
    0.000018255,
    0.00004573,
    0.00085328,
    0.000002476,
    0.0000038087,
    7.4908e-7,
    0.0000019515,
    0.000011862,
    6.5501e-7,
    0.0000047619,
    0.000019913,
    0.000011968,
    0.0000034634,
    0.000012992,
    0.000010003,
    0.0000014798,
    0.0000031936,
    0.0000017562,
    0.0000051596,
    0.00001699,
    0.0000031397,
    0.000016491,
    0.0000011977,
    0.0000015578,
    1.7911e-7,
    0.0000020153,
    0.044209,
    0.0000011983,
    0.000001268,
    0.000014544,
    9.9293e-7,
    7.3351e-7,
    8.6798e-7,
    7.8372e-7,
    0.18823,
    0.00004403,
    0.0000026206,
    0.000011821,
    0.0000015904,
    0.044916,
    0.000012113,
    9.2079e-7,
    0.0066987,
    0.0000012565,
    0.0000044158,
    0.000028145,
    0.000028259,
    0.0000036962,
    8.5015e-7,
    5.0633e-7,
    0.000027078,
    9.7998e-7,
    0.0000020004,
    0.0000030672,
    0.0008127,
    0.000012934,
    0.000015569,
    0.00023026,
    0.0000015524,
    0.45121,
    0.0010231,
    0.0000033744,
    7.8276e-7,
    0.0000018979,
    0.0000066964,
    0.0000027837,
    0.0000014946 ],
  [ 0.00054867,
    0.0000090438,
    0.0011897,
    0.0000011315,
    0.0000010186,
    0.0000023804,
    8.7925e-7,
    0.00021962,
    1.3397e-7,
    0.000014057,
    0.0000024689,
    5.6981e-7,
    0.20971,
    0.0000092435,
    0.0000097276,
    0.0011054,
    0.054614,
    0.0000013194,
    0.0000020295,
    3.9917e-7,
    0.0000010399,
    0.0000063208,
    3.4904e-7,
    0.0000025375,
    0.0073619,
    0.0000063773,
    0.001191,
    0.10249,
    0.00011344,
    7.8853e-7,
    0.0000017018,
    9.3581e-7,
    0.0000027493,
    0.012225,
    0.000001673,
    0.002279,
    6.3822e-7,
    8.3014e-7,
    9.544e-8,
    0.0000010739,
    0.12247,
    6.3853e-7,
    6.7567e-7,
    0.31113,
    5.2911e-7,
    3.9087e-7,
    4.6253e-7,
    4.1761e-7,
    0.064629,
    0.0069421,
    0.0000013964,
    0.0000062989,
    8.4743e-7,
    0.040907,
    0.0000064549,
    4.9065e-7,
    0.0000021151,
    6.6957e-7,
    0.0000023531,
    0.00066364,
    0.000015059,
    0.00011007,
    4.5302e-7,
    2.698e-7,
    0.030717,
    5.2221e-7,
    0.000001066,
    0.0000016344,
    6.4898e-7,
    0.000115,
    0.0033595,
    0.012555,
    8.2722e-7,
    0.0002296,
    0.0000046479,
    0.0000017981,
    4.1711e-7,
    0.0000010113,
    0.011138,
    0.0018393,
    7.9642e-7 ],
  [ 0.061411,
    0.0026526,
    4.124e-7,
    8.1458e-7,
    7.333e-7,
    0.0027256,
    6.3298e-7,
    0.0000024574,
    9.6449e-8,
    0.00001012,
    0.0000017774,
    4.1022e-7,
    0.12716,
    0.0000066545,
    0.0010187,
    0.0041422,
    0.000093857,
    9.4987e-7,
    0.0000014611,
    2.8736e-7,
    7.4862e-7,
    0.0000045504,
    2.5127e-7,
    0.0000018267,
    0.0000076389,
    0.0081763,
    0.0000013286,
    0.0000049843,
    0.0000038374,
    5.6767e-7,
    0.0000012251,
    6.737e-7,
    0.0000019793,
    0.00031782,
    0.0000012044,
    0.000084151,
    4.5946e-7,
    5.9761e-7,
    6.8709e-8,
    7.731e-7,
    0.021784,
    4.5968e-7,
    4.8641e-7,
    0.004286,
    0.00093428,
    2.8139e-7,
    3.3297e-7,
    3.0065e-7,
    0.3446,
    0.042821,
    0.0000010053,
    0.0056079,
    6.1007e-7,
    0.016919,
    0.0000046468,
    3.5323e-7,
    0.0070058,
    4.8202e-7,
    0.0000016939,
    0.33715,
    0.0070151,
    0.0000014179,
    3.2613e-7,
    1.9424e-7,
    0.000010388,
    3.7593e-7,
    7.6739e-7,
    0.0000011766,
    4.672e-7,
    0.0000049616,
    0.0000059727,
    0.000010507,
    5.9552e-7,
    0.0000096421,
    0.0039724,
    0.0000012945,
    3.0028e-7,
    7.2806e-7,
    0.0000025688,
    0.0000010679,
    5.7335e-7 ] ])

In [ ]:
(topicMatrix[1] == topicMatrix2[1]).sum()

In [ ]:
t1 = np.array([ 0.00054867,
  0.0000090438,
  0.0011897,
  0.0000011315,
  0.0000010186,
  0.0000023804,
  8.7925e-7,
  0.00021962,
  1.3397e-7,
  0.000014057,
  0.0000024689,
  5.6981e-7,
  0.20971,
  0.0000092435,
  0.0000097276,
  0.0011054,
  0.054614,
  0.0000013194,
  0.0000020295,
  3.9917e-7,
  0.0000010399,
  0.0000063208,
  3.4904e-7,
  0.0000025375,
  0.0073619,
  0.0000063773,
  0.001191,
  0.10249,
  0.00011344,
  7.8853e-7,
  0.0000017018,
  9.3581e-7,
  0.0000027493,
  0.012225,
  0.000001673,
  0.002279,
  6.3822e-7,
  8.3014e-7,
  9.544e-8,
  0.0000010739,
  0.12247,
  6.3853e-7,
  6.7567e-7,
  0.31113,
  5.2911e-7,
  3.9087e-7,
  4.6253e-7,
  4.1761e-7,
  0.064629,
  0.0069421,
  0.0000013964,
  0.0000062989,
  8.4743e-7,
  0.040907,
  0.0000064549,
  4.9065e-7,
  0.0000021151,
  6.6957e-7,
  0.0000023531,
  0.00066364,
  0.000015059,
  0.00011007,
  4.5302e-7,
  2.698e-7,
  0.030717,
  5.2221e-7,
  0.000001066,
  0.0000016344,
  6.4898e-7,
  0.000115,
  0.0033595,
  0.012555,
  8.2722e-7,
  0.0002296,
  0.0000046479,
  0.0000017981,
  4.1711e-7,
  0.0000010113,
  0.011138,
  0.0018393,
  7.9642e-7 ])

In [ ]:
t2 = np.array([ 0.00054867,
  0.0000090438,
  0.0011897,
  0.0000011315,
  0.0000010186,
  0.0000023804,
  8.7925e-7,
  0.00021962,
  1.3397e-7,
  0.000014057,
  0.0000024689,
  5.6981e-7,
  0.20971,
  0.0000092435,
  0.0000097276,
  0.0011054,
  0.054614,
  0.0000013194,
  0.0000020295,
  3.9917e-7,
  0.0000010399,
  0.0000063208,
  3.4904e-7,
  0.0000025375,
  0.0073619,
  0.0000063773,
  0.001191,
  0.10249,
  0.00011344,
  7.8853e-7,
  0.0000017018,
  9.3581e-7,
  0.0000027493,
  0.012225,
  0.000001673,
  0.002279,
  6.3822e-7,
  8.3014e-7,
  9.544e-8,
  0.0000010739,
  0.12247,
  6.3853e-7,
  6.7567e-7,
  0.31113,
  5.2911e-7,
  3.9087e-7,
  4.6253e-7,
  4.1761e-7,
  0.064629,
  0.0069421,
  0.0000013964,
  0.0000062989,
  8.4743e-7,
  0.040907,
  0.0000064549,
  4.9065e-7,
  0.0000021151,
  6.6957e-7,
  0.0000023531,
  0.00066364,
  0.000015059,
  0.00011007,
  4.5302e-7,
  2.698e-7,
  0.030717,
  5.2221e-7,
  0.000001066,
  0.0000016344,
  6.4898e-7,
  0.000115,
  0.0033595,
  0.012555,
  8.2722e-7,
  0.0002296,
  0.0000046479,
  0.0000017981,
  4.1711e-7,
  0.0000010113,
  0.011138,
  0.0018393,
  7.9642e-7 ])

In [ ]:
(t1==t2).sum()

In [ ]:
cuisineRelatedTopics = [0, 1, 3, 4, 6, 8, 10, 13, 16, 18, 25, 26, 27, 28, 30, 35, 38, 40, 42, 47, 51, 52, 54, 56, 59, 61, 62, 66, 67, 68, 71, 74, 75, 83, 84, 86, 90, 91, 92, 93, 96, 101, 106, 107, 108, 111, 114, 119, 122, 123, 124, 125, 126, 128, 129, 131, 132, 133, 134, 135, 136, 139, 140, 142, 143, 144, 146, 147, 148, 149, 150, 154, 157, 160, 163, 164, 165, 166, 169, 170, 171, 173, 176];

noncuisineRelatedTopics = [9, 11,14,19,94, 21, 29, 33, 45, 46, 50, 58, 70, 72, 88, 89, 97, 99, 100, 104, 105, 109, 110, 113, 116, 117,120, 141, 152, 158, 162, 168, 177, 179];



topicMerges = {
                        42:59,
                        107:59,
                        160:59,
                        0:59,
                        4:132,
                        164:132,
                        83:132,
                        84:132,
                        6:157,
                        38:157,
                        8:52,
                        106:10,
                        13:101,
                        150:101,
                        62:25,
                        26:30,
                        108:114,
122:114,
139:114,
147:114,
61:114,
67:166,
136:166,
68:166,
40:166,
111:166,
28:166,
169:166,
124:90,
93:90,
135:90,
106:92,
146:123,
163:123,
125:143,
154:131,
140:86
               };


In [ ]:
len(cuisineRelatedTopics)+len(noncuisineRelatedTopics)-len(topicMerges)

In [ ]:
len(cuisineRelatedTopics)

In [ ]:
len(noncuisineRelatedTopics)

In [ ]:
list(topicMerges.keys())

In [ ]:
cn = cuisineRelatedTopics + noncuisineRelatedTopics 

In [ ]:
finallist = []
for item in cn:
    if item not in list(topicMerges.keys()):
        finallist.append(item)

In [ ]:
finallist

In [ ]:
len(finallist)

In [ ]:
first =  [ 'service_1549651286734i3ucnks43ci',
        'service_1549650129390tei537ea35',
        'service_15496510966848rowym4hzd3',
        'service_1549649911396wgljgll3ctb',
        'service_15496502816047xd3iztxfqd',
        'service_1549650292002mrap5m84w6',
        'service_15496498235721j9cpi8s9fz',
        'service_1549649981031umr46sl4lkl',
        'service_1549650210930b92i6twni16',
        'service_1549650090192qess8jvxz',
        'service_154965029200250d9zn0k2rf',
        'service_1549649863837n7rqyxtdmsr',
        'service_1549650575479dpoc781utrt',
        'service_1549650238790kpl96wnmxhr',
        'service_1549650011117h5h7tkc7md4',
        'service_15496502619990tdhvj7zo0d9',
        'service_1549650185304zl1pffafs0b',
        'service_1549649931034j7ok2t0cd09',
        'service_15496504585920rn6530t2m89',
        'service_1549649875530j6ouyl7odwm',
        'service_1549649823572asg0k218ix5',
        'service_1549650339717vj0k1a6z45r',
        'service_1549649813759ny058f0fscq',
        'service_1549650090192tpckqqj0etq',
        'service_1549650011117sd9ggwd7e7',
        'service_1549650628041kywtqdlb4fk',
        'service_1549650303582wvgims9d9sp',
        'service_1549650782859rdiy041ouip',
        'service_1549650314765749n96rzpa4',
        'service_1549649911396cbpx9oudge',
        'service_1549650011117lrc1ctb4q3c',
        'service_1549651189282gie8nk29j2c',
        'service_15496498638379ssr0wr32i',
        'service_1549649834249urd8hhstwio',
        'service_1549650198061tx5p2jr6qfp',
        'service_1549650506785zrqww0e7z6',
        'service_1549650198061qu6nkomebk',
        'service_1549650021838izdwpqmdxpg',
        'service_1549651626318i06f5gqd32r',
        'service_1549649981031ietdhw4tg5n',
        'service_1549650281604lqyhmh7qm8a',
        'service_1549650470974ng03z0l9b2g',
        'service_1549650021838t1nnt6nh09',
        'service_1549650327042sfj6117m2pa',
        'service_1549649834249yqhu01wg4k',
        'service_154965023879089hcvzui7i',
        'service_15496508830373fs4t6740ih',
        'service_15496500901927iifuv0wkm',
        'service_1549650272498k8kj2be1y7',
        'service_1549650238790gwuzdsvuedt',
        'service_1549650314765qnnu7u0kb6m',
        'service_15496504834224ix8s5co19o',
        'service_15496504020759mb74o9gmur',
        'service_1549650303582o3krgsbwxna',
        'service_154965027249822rtw8ckmbb',
        'service_15496503270429wrgmr5domg',
        'service_1549649954848uxkna1mdf0c',
        'service_1549649990473xcm9x3oi2v',
        'service_1549650155208qntuqzfjp39',
        'service_1549650549747ywmnac505vo',
        'service_154965011666208ykql2w1j5d',
        'service_1549650351146qjsvn0mjr9q',
        'service_15496499681925x64n52mbqd',
        'service_1549649899477ck7l55v9ry8',
        'service_1549650261999lttonk96gzm',
        'service_1549650090192c80dch29c4g',
        'service_15496498755308e0lt577nnf',
        'service_1549650223756e97p69fajjb',
        'service_1549650782859y5tk07jv96h',
        'service_1549649968192n92p3vcidt',
        'service_1549650446540nco1q389ois',
        'service_1549650155208kh43tfrzv8n',
        'service_1549650388839kvgt6secrb',
        'service_1549650011117d4ywhsbbgpt',
        'service_1549649813759wn8jbjq64m',
        'service_1549650210930szufk9jrsdq',
        'service_1549649854375pj40ofwy2g',
        'service_1549650079653cuvde2y1ill',
        'service_1549650251199xdjsojucyo',
        'service_15496502816044pkg4abvaii',
        'service_15496508075779d5asrzzfii',
        'service_1549650021838znng9m6pmur',
        'service_1549650090192tq95g6kibbn',
        'service_1549650303582zkmd4bjpdu',
        'service_15496499810310q3mya82uhfe',
        'service_1549650339717rlcwcih084l',
        'service_1549649834249rgjl3jlzgyd',
        'service_1549649899477u7b9h1xnk7p',
        'service_1549649931034wpkino2tjis',
        'service_1549650100958aq23q6vtcmn',
        'service_1549649844546gzpib8ht7k7',
        'service_1549649875530z9s14d1rpmb',
        'service_1549649941527jxlj7cz5kh',
        'service_1549650021838y4ageh8gb1p',
        'service_1549649981031va1rju728u',
        'service_1549649899477ltbjb5194oh',
        'service_1549650046511zp5g0p0x128',
        'service_1549649941527qgwmdpfcuk',
        'service_1549650155208k4hgxl4rc4',
        'service_1549649981031aw2m3carn1']

In [ ]:
second = [ 'service_1549651286734i3ucnks43ci',
        'service_1549650129390tei537ea35',
        'service_15496510966848rowym4hzd3',
        'service_1549649911396wgljgll3ctb',
        'service_15496502816047xd3iztxfqd',
        'service_1549650292002mrap5m84w6',
        'service_15496498235721j9cpi8s9fz',
        'service_1549649981031umr46sl4lkl',
        'service_1549650210930b92i6twni16',
        'service_1549650090192qess8jvxz',
        'service_154965029200250d9zn0k2rf',
        'service_1549649863837n7rqyxtdmsr',
        'service_1549650575479dpoc781utrt',
        'service_1549650238790kpl96wnmxhr',
        'service_1549650011117h5h7tkc7md4',
        'service_15496502619990tdhvj7zo0d9',
        'service_1549650185304zl1pffafs0b',
        'service_1549649931034j7ok2t0cd09',
        'service_15496504585920rn6530t2m89',
        'service_1549649875530j6ouyl7odwm',
        'service_1549649823572asg0k218ix5',
        'service_1549650339717vj0k1a6z45r',
        'service_1549649813759ny058f0fscq',
        'service_1549650090192tpckqqj0etq',
        'service_1549650011117sd9ggwd7e7',
        'service_1549650628041kywtqdlb4fk',
        'service_1549650303582wvgims9d9sp',
        'service_1549650782859rdiy041ouip',
        'service_1549650314765749n96rzpa4',
        'service_1549649911396cbpx9oudge',
        'service_1549650011117lrc1ctb4q3c',
        'service_1549651189282gie8nk29j2c',
        'service_15496498638379ssr0wr32i',
        'service_1549649834249urd8hhstwio',
        'service_1549650198061tx5p2jr6qfp',
        'service_1549650506785zrqww0e7z6',
        'service_1549650198061qu6nkomebk',
        'service_1549650021838izdwpqmdxpg',
        'service_1549651626318i06f5gqd32r',
        'service_1549649981031ietdhw4tg5n',
        'service_1549650281604lqyhmh7qm8a',
        'service_1549650470974ng03z0l9b2g',
        'service_1549650021838t1nnt6nh09',
        'service_1549650327042sfj6117m2pa',
        'service_1549649834249yqhu01wg4k',
        'service_154965023879089hcvzui7i',
        'service_15496508830373fs4t6740ih',
        'service_15496500901927iifuv0wkm',
        'service_1549650272498k8kj2be1y7',
        'service_1549650238790gwuzdsvuedt',
        'service_1549650314765qnnu7u0kb6m',
        'service_15496504834224ix8s5co19o',
        'service_15496504020759mb74o9gmur',
        'service_1549650303582o3krgsbwxna',
        'service_154965027249822rtw8ckmbb',
        'service_15496503270429wrgmr5domg',
        'service_1549649954848uxkna1mdf0c',
        'service_1549649990473xcm9x3oi2v',
        'service_1549650155208qntuqzfjp39',
        'service_1549650549747ywmnac505vo',
        'service_154965011666208ykql2w1j5d',
        'service_1549650351146qjsvn0mjr9q',
        'service_15496499681925x64n52mbqd',
        'service_1549649899477ck7l55v9ry8',
        'service_1549650261999lttonk96gzm',
        'service_1549650090192c80dch29c4g',
        'service_15496498755308e0lt577nnf',
        'service_1549650223756e97p69fajjb',
        'service_1549650782859y5tk07jv96h',
        'service_1549649968192n92p3vcidt',
        'service_1549650446540nco1q389ois',
        'service_1549650155208kh43tfrzv8n',
        'service_1549650388839kvgt6secrb',
        'service_1549650011117d4ywhsbbgpt',
        'service_1549649813759wn8jbjq64m',
        'service_1549650210930szufk9jrsdq',
        'service_1549649854375pj40ofwy2g',
        'service_1549650079653cuvde2y1ill',
        'service_1549650251199xdjsojucyo',
        'service_15496502816044pkg4abvaii',
        'service_15496508075779d5asrzzfii',
        'service_1549650021838znng9m6pmur',
        'service_1549650090192tq95g6kibbn',
        'service_1549650303582zkmd4bjpdu',
        'service_15496499810310q3mya82uhfe',
        'service_1549650339717rlcwcih084l',
        'service_1549649834249rgjl3jlzgyd',
        'service_1549649899477u7b9h1xnk7p',
        'service_1549649931034wpkino2tjis',
        'service_1549650100958aq23q6vtcmn',
        'service_1549649844546gzpib8ht7k7',
        'service_1549649875530z9s14d1rpmb',
        'service_1549649941527jxlj7cz5kh',
        'service_1549650021838y4ageh8gb1p',
        'service_1549649981031va1rju728u',
        'service_1549649899477ltbjb5194oh',
        'service_1549650046511zp5g0p0x128',
        'service_1549649941527qgwmdpfcuk',
        'service_1549650155208k4hgxl4rc4',
        'service_1549649981031aw2m3carn1']

In [ ]:
first == second

In [ ]:
len(second)

In [ ]:
len( [ 'service_1549650129390tei537ea35',
        'service_15496510966848rowym4hzd3',
        'service_1549649911396wgljgll3ctb',
        'service_15496498235721j9cpi8s9fz',
        'service_1549649981031umr46sl4lkl',
        'service_1549650210930b92i6twni16',
        'service_1549650090192qess8jvxz',
        'service_154965029200250d9zn0k2rf',
        'service_1549649863837n7rqyxtdmsr',
        'service_1549650575479dpoc781utrt',
        'service_1549650238790kpl96wnmxhr',
        'service_15496502619990tdhvj7zo0d9',
        'service_1549650185304zl1pffafs0b',
        'service_1549649931034j7ok2t0cd09',
        'service_15496504585920rn6530t2m89',
        'service_1549649875530j6ouyl7odwm',
        'service_1549649823572asg0k218ix5',
        'service_1549650339717vj0k1a6z45r',
        'service_1549650090192tpckqqj0etq',
        'service_1549650628041kywtqdlb4fk',
        'service_1549650303582wvgims9d9sp',
        'service_1549650782859rdiy041ouip',
        'service_1549650314765749n96rzpa4',
        'service_1549649911396cbpx9oudge',
        'service_1549650011117lrc1ctb4q3c',
        'service_1549651189282gie8nk29j2c',
        'service_1549649834249urd8hhstwio',
        'service_1549650198061tx5p2jr6qfp',
        'service_1549650506785zrqww0e7z6',
        'service_1549650198061qu6nkomebk',
        'service_1549650021838izdwpqmdxpg',
        'service_1549649981031ietdhw4tg5n',
        'service_1549650281604lqyhmh7qm8a',
        'service_1549650470974ng03z0l9b2g',
        'service_1549650021838t1nnt6nh09',
        'service_1549650327042sfj6117m2pa',
        'service_1549649834249yqhu01wg4k',
        'service_154965023879089hcvzui7i',
        'service_15496508830373fs4t6740ih',
        'service_15496500901927iifuv0wkm',
        'service_1549650272498k8kj2be1y7',
        'service_1549650238790gwuzdsvuedt',
        'service_1549650314765qnnu7u0kb6m',
        'service_15496504834224ix8s5co19o',
        'service_15496504020759mb74o9gmur',
        'service_1549650303582o3krgsbwxna',
        'service_154965027249822rtw8ckmbb',
        'service_15496503270429wrgmr5domg',
        'service_1549649954848uxkna1mdf0c',
        'service_1549649990473xcm9x3oi2v',
        'service_1549650155208qntuqzfjp39',
        'service_154965011666208ykql2w1j5d',
        'service_15496499681925x64n52mbqd',
        'service_1549650261999lttonk96gzm',
        'service_1549650090192c80dch29c4g',
        'service_15496498755308e0lt577nnf',
        'service_1549650223756e97p69fajjb',
        'service_1549650446540nco1q389ois',
        'service_1549650155208kh43tfrzv8n',
        'service_1549650011117d4ywhsbbgpt',
        'service_15496502816044pkg4abvaii',
        'service_15496508075779d5asrzzfii',
        'service_1549650021838znng9m6pmur',
        'service_1549650090192tq95g6kibbn',
        'service_1549650303582zkmd4bjpdu',
        'service_15496499810310q3mya82uhfe',
        'service_1549650339717rlcwcih084l',
        'service_1549650100958aq23q6vtcmn',
        'service_1549649875530z9s14d1rpmb',
        'service_1549649941527jxlj7cz5kh',
        'service_1549650021838y4ageh8gb1p',
        'service_1549649899477ltbjb5194oh',
        'service_1549650046511zp5g0p0x128',
        'service_1549649941527qgwmdpfcuk',
        'service_1549650155208k4hgxl4rc4',
        'service_1549649981031aw2m3carn1',
        'service_1549649941527ywf4nk9cs8d',
        'service_1549649911396hlz159h74c7',
        'service_15496499415275fuahnz7tu7',
        'service_1549649920795n9vjhy8le1n',
        'service_1549649954848q126rv6nq1',
        'service_1549649931034o2jw9dq4ofh',
        'service_1549650062205w9rndsu40d',
        'service_1549649911396uigtdmfs3qm',
        'service_1549650223757q4yyqrd2df',
        'service_1549649875530f3c20teaasq',
        'service_1549650261999zup2jw1sfy',
        'service_1549649875530i61quec7u8b',
        'service_1549649887884bbakprn88iq' ])

In [ ]:
len([ 'service_1549651286734i3ucnks43ci',
        'service_1549650129390tei537ea35',
        'service_15496510966848rowym4hzd3',
        'service_1549649911396wgljgll3ctb',
        'service_15496502816047xd3iztxfqd',
        'service_1549650292002mrap5m84w6',
        'service_15496498235721j9cpi8s9fz',
        'service_1549649981031umr46sl4lkl',
        'service_1549650210930b92i6twni16',
        'service_1549650090192qess8jvxz',
        'service_154965029200250d9zn0k2rf',
        'service_1549649863837n7rqyxtdmsr',
        'service_1549650575479dpoc781utrt',
        'service_1549650238790kpl96wnmxhr',
        'service_1549650011117h5h7tkc7md4',
        'service_15496502619990tdhvj7zo0d9',
        'service_1549650185304zl1pffafs0b',
        'service_1549649931034j7ok2t0cd09',
        'service_15496504585920rn6530t2m89',
        'service_1549649875530j6ouyl7odwm',
        'service_1549649823572asg0k218ix5',
        'service_1549650339717vj0k1a6z45r',
        'service_1549649813759ny058f0fscq',
        'service_1549650090192tpckqqj0etq',
        'service_1549650011117sd9ggwd7e7',
        'service_1549650628041kywtqdlb4fk',
        'service_1549650303582wvgims9d9sp',
        'service_1549650782859rdiy041ouip',
        'service_1549650314765749n96rzpa4',
        'service_1549649911396cbpx9oudge',
        'service_1549650011117lrc1ctb4q3c',
        'service_1549651189282gie8nk29j2c',
        'service_15496498638379ssr0wr32i',
        'service_1549649834249urd8hhstwio',
        'service_1549650198061tx5p2jr6qfp',
        'service_1549650506785zrqww0e7z6',
        'service_1549650198061qu6nkomebk',
        'service_1549650021838izdwpqmdxpg',
        'service_1549651626318i06f5gqd32r',
        'service_1549649981031ietdhw4tg5n',
        'service_1549650281604lqyhmh7qm8a',
        'service_1549650470974ng03z0l9b2g',
        'service_1549650021838t1nnt6nh09',
        'service_1549650327042sfj6117m2pa',
        'service_1549649834249yqhu01wg4k',
        'service_154965023879089hcvzui7i',
        'service_15496508830373fs4t6740ih',
        'service_15496500901927iifuv0wkm',
        'service_1549650272498k8kj2be1y7',
        'service_1549650238790gwuzdsvuedt',
        'service_1549650314765qnnu7u0kb6m',
        'service_15496504834224ix8s5co19o',
        'service_15496504020759mb74o9gmur',
        'service_1549650303582o3krgsbwxna',
        'service_154965027249822rtw8ckmbb',
        'service_15496503270429wrgmr5domg',
        'service_1549649954848uxkna1mdf0c',
        'service_1549649990473xcm9x3oi2v',
        'service_1549650155208qntuqzfjp39',
        'service_1549650549747ywmnac505vo',
        'service_154965011666208ykql2w1j5d',
        'service_1549650351146qjsvn0mjr9q',
        'service_15496499681925x64n52mbqd',
        'service_1549649899477ck7l55v9ry8',
        'service_1549650261999lttonk96gzm',
        'service_1549650090192c80dch29c4g',
        'service_15496498755308e0lt577nnf',
        'service_1549650223756e97p69fajjb',
        'service_1549650782859y5tk07jv96h',
        'service_1549649968192n92p3vcidt',
        'service_1549650446540nco1q389ois',
        'service_1549650155208kh43tfrzv8n',
        'service_1549650388839kvgt6secrb',
        'service_1549650011117d4ywhsbbgpt',
        'service_1549649813759wn8jbjq64m',
        'service_1549650210930szufk9jrsdq',
        'service_1549649854375pj40ofwy2g',
        'service_1549650079653cuvde2y1ill',
        'service_1549650251199xdjsojucyo',
        'service_15496502816044pkg4abvaii',
        'service_15496508075779d5asrzzfii',
        'service_1549650021838znng9m6pmur',
        'service_1549650090192tq95g6kibbn',
        'service_1549650303582zkmd4bjpdu',
        'service_15496499810310q3mya82uhfe',
        'service_1549650339717rlcwcih084l',
        'service_1549649834249rgjl3jlzgyd',
        'service_1549649899477u7b9h1xnk7p',
        'service_1549649931034wpkino2tjis',
        'service_1549650100958aq23q6vtcmn',
        'service_1549649844546gzpib8ht7k7',
        'service_1549649875530z9s14d1rpmb',
        'service_1549649941527jxlj7cz5kh',
        'service_1549650021838y4ageh8gb1p',
        'service_1549649981031va1rju728u',
        'service_1549649899477ltbjb5194oh',
        'service_1549650046511zp5g0p0x128',
        'service_1549649941527qgwmdpfcuk',
        'service_1549650155208k4hgxl4rc4',
        'service_1549649981031aw2m3carn1'])